# Imports

In [1]:
# Core imports
import numpy as np
import pandas as pd
import logging
import os
import random
import time
import json
import pickle

# Machine learning imports
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    StandardScaler, OneHotEncoder, LabelEncoder, TargetEncoder
)

# Deep learning imports
import torch
import torch.nn.functional as F
import tensorflow as tf

# Model-specific imports
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from tabpfn import TabPFNClassifier, TabPFNRegressor
from GRANDE import GRANDE
from packages.tabm_reference import Model, make_parameter_groups

# Optimization imports
import optuna

# Data source imports
import openml

# Abstract base class imports
from abc import ABC, abstractmethod
from typing import Dict, Literal

2025-06-06 11:33:25.437384: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-06 11:33:26.852015: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mherre/.local/lib/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/home/mherre/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Utils

In [2]:
def set_seed(seed: int = 42) -> None:
    """
    Set random seeds for reproducibility across Python, NumPy, PyTorch, and TensorFlow.

    Args:
        seed: Integer seed for random number generators
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    tf.random.set_seed(seed)

def setup_logging() -> logging.Logger:
    """
    Set up logging configuration.
    This function configures the logging settings for the application, including
    the logging level and format. It returns a logger instance that can be used
    throughout the application.
    Returns:
        logging.Logger: Configured logger instance.
    """
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    )
    logger = logging.getLogger(__name__)
    return logger


def check_GPU_availability():
    """
    Checks if a GPU is available and configures TensorFlow to use it appropriately.

    Sets up memory growth for TensorFlow GPU usage to avoid allocating all GPU memory at once.

    Returns:
        torch.device: A torch device object ('cuda' if GPU is available, 'cpu' otherwise)
    """
    logger = setup_logging()
    if torch.cuda.is_available():
        logger.info(f"Using GPU: {torch.cuda.get_device_name()}")
        device = torch.device("cuda")
        gpus = tf.config.experimental.list_physical_devices("GPU")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        device = torch.device("cpu")
        logger.info("Using CPU for training.")
    return device

# Preprocessing

In [3]:
def encode_target(y):
    """
    Encode the target variable using LabelEncoder.

    Parameters
    ----------
    y : np.array
        The target variable to encode.

    Returns
    -------
    y_encoded : np.array
        The encoded target variable.
    """
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    return y_encoded


def preprocess(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_val: pd.DataFrame,
    cat_cols: list,
    config: dict,
    preprocessing_type: str,
) -> tuple:
    """
    Preprocess the training and validation datasets based on the specified model type.

    Parameters
    ----------
    X_train : pandas.DataFrame
        The training dataset features.
    y_train : pandas.Series or np.array
        The target variable for the training dataset.
    X_val : pandas.DataFrame
        The validation dataset features.
    cat_cols : list or array of bool
        Boolean mask indicating which columns in X_train and X_val are categorical (True).
    config : dict
        Configuration dictionary containing model and preprocessing details.

    Returns
    -------
    X_train : np.array
        The preprocessed training dataset features.
    X_val : np.array
        The preprocessed validation dataset features.
    """
    X_train, preprocessor_inner = _minimal_preprocess_train(
        X_train,
        y_train,
        cat_cols,
        preprocessing_type,
        config,
    )
    X_val = _minimal_preprocess_test(X_val, preprocessor_inner)

    return X_train, X_val


def _minimal_preprocess_train(X, y, categorical_features, model_type, config):
    """
    Perform minimal preprocessing on the input features for training.

    This function applies preprocessing to both numeric and categorical features
    based on the model type specified in the configuration. For Neural Networks,
    it standardizes numeric features and one-hot encodes categorical features. For
    tree-based models, it does not scale numeric features and applies different
    encoding strategies for low- and high-cardinality categorical features.

    Parameters
    ----------
    X : pandas.DataFrame
        The input features.

    categorical_features : list or array of bool
        Boolean mask indicating which columns in X are categorical (True) and
        which are numeric (False).

    Returns
    -------
    X_preprocessed : np.array
        The preprocessed features.
    preprocessor : ColumnTransformer
        The fitted preprocessor, which can be used to transform future datasets.
    """

    # Convert the boolean mask to a NumPy array (if not already) for fast indexing.
    cat_mask = np.asarray(categorical_features)

    # Check that the mask length matches the number of columns in X.
    if cat_mask.shape[0] != X.shape[1]:
        raise ValueError(
            "Length of categorical_features mask must match the number of columns in X."
        )

    # Extract column names using boolean indexing.
    categorical_feature_names = X.columns[cat_mask].tolist()
    numeric_feature_names = X.columns[~cat_mask].tolist()

    # Select pipeline based on config.
    logger = setup_logging()

    # Neural Networks: standardize numeric features and one-hot encode categoricals.
    if model_type == "nn":

        # Define the preprocessing pipeline for numeric features.
        numeric_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="mean")),
                ("scaler", StandardScaler()),
            ]
        )

        # Define the preprocessing pipeline for categorical features.
        categorical_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                (
                    "onehot",
                    OneHotEncoder(
                        drop="first", sparse_output=False, handle_unknown="ignore"
                    ),
                ),
            ]
        )

        transformers = [
            ("num", numeric_pipeline, numeric_feature_names),
            ("cat", categorical_pipeline, categorical_feature_names),
        ]

        logger.info("Preprocessing pipeline for Neural Network model.")

    elif model_type == "tree":

        # Tree-based models: leave numeric features unscaled, and differentiate encoding for categoricals.
        numeric_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="mean")),
            ]
        )
        # Split categorical features into low- and high-cardinality groups.
        threshold = config["preprocessing"]["threshold_high_cardinality"]
        low_card_features = []
        high_card_features = []
        for feature in categorical_feature_names:
            if X[feature].nunique() <= threshold:
                low_card_features.append(feature)
            else:
                high_card_features.append(feature)

        # For low-cardinality categoricals, use one-hot encoding.
        low_card_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                (
                    "onehot",
                    OneHotEncoder(
                        drop="first", sparse_output=False, handle_unknown="ignore"
                    ),
                ),
            ]
        )

        # For high-cardinality categoricals, use target encoding.
        high_card_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("target_encode", TargetEncoder()),
            ]
        )

        transformers = [
            ("num", numeric_pipeline, numeric_feature_names),
            ("low_card", low_card_pipeline, low_card_features),
            ("high_card", high_card_pipeline, high_card_features),
        ]

        logger.info("Preprocessing pipeline for Decision Tree model.")

    elif model_type == "grande":
        logger.info("No preprocessing required for Grande model.")
        return X, None

    else:
        raise ValueError(
            "Unknown teacher type. Must be 'nn' for Neural Network, 'tree' for Decision Tree, or 'grande' for Gradient Based Tree."
        )

    # Create a ColumnTransformer to apply the transformations to the appropriate columns.
    preprocessor = ColumnTransformer(
        transformers=transformers,
        n_jobs=-2,
    )

    # Fit the preprocessor on the training data and transform it.
    X_preprocessed = preprocessor.fit_transform(X, y)

    return X_preprocessed, preprocessor


def _minimal_preprocess_test(X, preprocessor):
    """
    Transform the input features according to the preprocessor fitted on the training data.

    Parameters
    ----------
    X : pandas.DataFrame
        The input features.
    preprocessor : ColumnTransformer
        The preprocessor fitted on the training data.

    Returns
    -------
    X_preprocessed : np.array
        The preprocessed features.
    """
    if preprocessor is None:
        return X
    return preprocessor.transform(X)


# Data Loader

In [4]:
def load_dataset(dataset_id: int, config: dict):
    """
    Loads an OpenML dataset based on its ID and processes it according to the task type.

    This function fetches the dataset using the _fetch_dataset helper, then processes
    the target variable based on whether the task is binary classification or regression.

    Parameters:
        dataset_id (int): The ID of the dataset on OpenML.
        config (dict): Configuration dictionary containing data paths and settings.

    Returns:
        tuple: A tuple containing:
            - X: The feature data (pandas DataFrame)
            - y: The processed target variable (encoded for binary classification or numpy array for regression)
            - cat_cols: List of booleans indicating which features are categorical
            - attribute_names: List of attribute names
            - task_type: String indicating the task type ('binary' or 'regression')
    """
    X, y, cat_cols, attribute_names, task_type = fetch_dataset(
        dataset_id=dataset_id,
        cache_dir=config["data"]["cache_dir_path"],
    )
    if task_type == "binary":
        # Encode the target variable if it's binary
        y = encode_target(y)
    else:
        # Transform to np.array for regression
        y = np.array(y, dtype=float)

    return X, y, cat_cols, attribute_names, task_type


def fetch_dataset(dataset_id: int, cache_dir: str = "data/cache/"):
    """
    Downloads an OpenML dataset based on its id, caches the data locally, and returns the data
    along with its metadata.

    The five returned objects are:
        - X: The feature data (typically a pandas DataFrame).
        - y: The target variable.
        - categorical_indicator: A list of booleans indicating which features are categorical.
        - attribute_names: A list of attribute names.
        - task_type: A string indicating the type of task ('binary' or 'regression').

    If the dataset has been previously downloaded and stored in the cache directory,
    it will be loaded from the local file instead of re-downloading.

    Parameters:
        dataset_id (int): The id of the dataset on OpenML.
        cache_dir (str): The directory to store the downloaded dataset. Defaults to "openml_cache".

    Returns:
        X, y, categorical_indicator, attribute_names, task_type
    """
    logger = setup_logging()

    # Ensure the cache directory exists
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir, exist_ok=True)

    # Define a cache file name that is unique to the dataset id
    cache_file = os.path.join(cache_dir, f"openml_dataset_{dataset_id}.pkl")

    # If the cache file exists, load the data from the file.
    if os.path.exists(cache_file):
        logger.info(
            f"Loading dataset {dataset_id} from cache at '{cache_file}'..."
        )
        with open(cache_file, "rb") as f:
            data = pickle.load(f)
        X = data["X"]
        y = data["y"]
        categorical_indicator = data["categorical_indicator"]
        attribute_names = data["attribute_names"]
        task_type = data["task_type"]
    else:
        # Download the dataset from OpenML.
        logger.info(f"Downloading dataset {dataset_id} from OpenML...")
        dataset = openml.datasets.get_dataset(dataset_id)

        # Use the default target attribute (if defined) when retrieving the data.
        X, y, categorical_indicator, attribute_names = dataset.get_data(
            target=dataset.default_target_attribute, dataset_format="dataframe"
        )

        # Determine the task type (binary classification or regression)
        task_type = _determine_task_type(y)

        # Store the data and metadata in a dictionary.
        data = {
            "X": X,
            "y": y,
            "categorical_indicator": categorical_indicator,
            "attribute_names": attribute_names,
            "task_type": task_type,
        }

        # Save the dictionary to a local file using pickle.
        with open(cache_file, "wb") as f:
            pickle.dump(data, f)
        logger.info(f"Dataset {dataset_id} stored locally at '{cache_file}'.")

    logger.info(f"Dataset {dataset_id} loaded successfully with task type: {task_type}")

    return X, y, categorical_indicator, attribute_names, task_type


def _determine_task_type(y):
    """
    Determines if the target variable is for binary classification, or regression.

    Parameters:
        y: The target variable (pandas dataframe).

    Returns:
        str: 'binary', or 'regression'
    """

    # Get unique values
    unique_values = pd.unique(y)

    # Check if it's binary (2 unique values)
    if len(unique_values) == 2:
        return "binary"
    else:
        return "regression"

# Evaluate

In [5]:
def evaluate_classification(y_prob, y_true):
    """
    Evaluate classification performance using balanced accuracy, F1 score, and ROC AUC.

    Parameters:
        y_prob (np.array): Predicted probabilities for the positive class.
        y_true (np.array): True labels.

    Returns:
        dict: Dictionary containing evaluation metrics.
    """
    logger = setup_logging()
    y_pred = (y_prob[:, 1] > 0.5).astype(int)
    acc = balanced_accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    roc_auc = roc_auc_score(y_true, y_prob[:, 1])

    logger.info(f"\t Balanced Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}")

    return {
        "acc": acc,
        "f1": f1,
        "roc": roc_auc,
    }

def evaluate_regression(y_pred, y_true):
    """
    Evaluate regression performance using R^2 score.

    Parameters:
        y_pred (np.array): Predicted values.
        y_true (np.array): True values.

    Returns:
        float: R^2 score.
    """
    logger = setup_logging()

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    logger.info(f"\t MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r2:.4f}")

    return {
        "mae": mae,
        "mse": mse,
        "rmse": rmse,
        "r2": r2,
    }

def evaluate_fidelity_classification(y_prob_student, y_prob_teacher):
    """
    Evaluate the fidelity of a student model compared to a teacher model in classification tasks.

    Parameters:
        y_prob_teacher (np.array): Predicted probabilities from the teacher model.
        y_prob_student (np.array): Predicted probabilities from the student model.

    Returns:
        dict: Dictionary containing evaluation metrics for fidelity.
    """
    logger = setup_logging()

    y_pred_student = (y_prob_student[:, 1] > 0.5).astype(int)

    if y_prob_teacher.ndim == 2:
        y_pred_teacher = (y_prob_teacher[:, 1] > 0.5).astype(int)
        kl_div = kl_divergence_soft(y_prob_student[:, 1], y_prob_teacher[:, 1])

    elif y_prob_teacher.ndim == 1:
        y_pred_teacher = y_prob_teacher.astype(int)
        kl_div = kl_divergence_hard(y_prob_student[:, 1], y_pred_teacher)

    else:
        raise ValueError(
            f"y_prob_teacher has unexpected shape: {y_prob_teacher.shape}"
        )
    
    acc = balanced_accuracy_score(y_pred_teacher, y_pred_student)
    f1 = f1_score(y_pred_teacher, y_pred_student, average="macro")
    roc_auc = roc_auc_score(y_pred_teacher, y_prob_student[:, 1])

    logger.info(f"\t Fidelity - Balanced Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}, KL Divergence: {kl_div:.4f}")

    return {
        "fidelity_acc": acc,
        "fidelity_f1": f1,
        "fidelity_roc": roc_auc,
        "fidelity_kl_div": kl_div,
    }

def evaluate_fidelity_regression(y_pred_student, y_pred_teacher):
    """
    Evaluate the fidelity of a student model compared to a teacher model in regression tasks.

    Parameters:
        y_pred_teacher (np.array): Predicted values from the teacher model.
        y_pred_student (np.array): Predicted values from the student model.

    Returns:
        dict: Dictionary containing evaluation metrics for fidelity.
    """
    logger = setup_logging()

    mae = mean_absolute_error(y_pred_teacher, y_pred_student)
    mse = mean_squared_error(y_pred_teacher, y_pred_student)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_pred_teacher, y_pred_student)

    logger.info(f"\t Fidelity - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r2:.4f}")

    return {
        "fidelity_mae": mae,
        "fidelity_mse": mse,
        "fidelity_rmse": rmse,
        "fidelity_r2": r2,
    }

def kl_divergence_hard(probs_pos, y_true):
    eps = 1e-7  
    probs_pos = np.clip(probs_pos, eps, 1 - eps)
    
    log_likelihood = y_true * np.log(probs_pos) + (1 - y_true) * np.log(1 - probs_pos)
    result = -np.mean(log_likelihood)
    
    return result if not np.isnan(result) else 0.0

def kl_divergence_soft(p, q):
    eps = 1e-7
    p = np.clip(p, eps, 1 - eps)
    q = np.clip(q, eps, 1 - eps)
    return np.mean(p * np.log(p / q) + (1 - p) * np.log((1 - p) / (1 - q)))

# Model Factory

## Teacher

In [ ]:
class TeacherModelBase(ABC):
    def __init__(self, **kwargs):
        self.params = kwargs

    @abstractmethod
    def train(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Trains the model on the given data.

        Parameters:
            X (np.ndarray): The input features with shape (n_samples, n_features).
            y (np.ndarray): The target labels with shape (n_samples,).

        Returns:
            None
        """
        pass

    @abstractmethod
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Makes predictions on the provided data.

        Parameters:
            X (np.ndarray): The input features with shape (n_samples, n_features).

        Returns:
            np.ndarray: An array of predicted probabilities with shape (n_samples, n_classes).
                        (If the model returns logits, these should be post-processed to probabilities.)
        """
        pass

    def evaluate(self, y_pred: np.ndarray, y_true: np.ndarray) -> Dict[str, float]:
        """
        Evaluates the model's predictions against the true labels.

        Parameters:
            y_pred (np.ndarray): The predicted probabilities or values.
            y_true (np.ndarray): The true labels.

        Returns:
            Dict[str, float]: A dictionary containing evaluation metrics.
        """
        if self.task_type == "binary":
            metrics = evaluate_classification(y_pred, y_true)
            metrics['parameters'] = self.count_parameters()
            return metrics
        
        elif self.task_type == "regression":
            metrics = evaluate_regression(y_pred, y_true)
            metrics['parameters'] = self.count_parameters()
            return metrics
        
        else:
            raise ValueError(f"Unknown task type: {self.task_type}. Must be 'binary' or 'regression'.")

    @abstractmethod
    def count_parameters(self) -> int:
        """
        Counts the number of trainable parameters in the model.

        Returns:
            int: The total number of trainable parameters.
        """
        pass


class TabPFNTeacherModel(TeacherModelBase):

    def __init__(self, **kwargs):
        """
        Initialize a TabPFN teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base
            class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        super().__init__(**kwargs)

    def train(self, X, y, **training_params):
        if self.task_type == "binary":
            self.model = TabPFNClassifier()
        else:
            self.model = TabPFNRegressor()
        # If X is bigger than 10000 samples, reduce data size to 10000
        if X.shape[0] > 10000:
            X = X[:10000]
            y = y[:10000]
        self.model.fit(X, y)

    def predict(self, X):
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self) -> int:
        """
        Counts the number of trainable parameters in the underlying TabPFN model.

        Returns:
            int: Total number of trainable parameters.

        Raises:
            ValueError: If the TabPFN model has not been fitted yet.
        """
        if not hasattr(self.model, "model_"):
            raise ValueError(
                "The TabPFN model is not fitted yet. Please call fit() first."
            )
        return sum(p.numel() for p in self.model.model_.parameters() if p.requires_grad)
    

class CatBoostTeacherModel(TeacherModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a CatBoost teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

    def train(self, X, y):
        """
        Train the CatBoost model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the CatBoost model's fit method.
        """
        config = self.config
        random_state = config["training"]["random_state"]

        # Initialize the CatBoost model
        if self.task_type == "binary":
            self.model = CatBoostClassifier(
                random_seed=random_state,
                thread_count=-1,  # Use all available CPU cores
                logging_level="Silent",  # Suppress CatBoost output
                task_type=("GPU" if str(self.device).lower() == "cuda" else "CPU"),
                **self.hyperparams,  # Include the sampled hyperparameters
            )
        else:
            self.model = CatBoostRegressor(
                random_seed=random_state,
                thread_count=-1,  # Use all available CPU cores
                logging_level="Silent",  # Suppress CatBoost output
                task_type=("GPU" if str(self.device).lower() == "cuda" else "CPU"),
                **self.hyperparams,  # Include the sampled hyperparameters
            )

        # Split data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )
        # Train the model
        self.model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=20, 
        )

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples, n_classes)
            The predicted probabilities for each class.
        """
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self):
        """
        Count the number of parameters in the trained CatBoost model.

        Returns
        -------
        int
            The number of parameters in the model.
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("The model has not been trained yet.")

        # Get the number of trees in the model
        tree_count = self.model.tree_count_

        # For each tree, count:
        # - split values (one per non-leaf node)
        # - leaf values (one per leaf node)
        # - feature indices (one per non-leaf node)
        # For a binary tree with depth d, there are 2^d - 1 non-leaf nodes and 2^d leaf nodes
        # This is a simplified approximation
        approx_depth = self.hyperparams.get("max_depth", 6)  # Default depth in CatBoost is 6
        non_leaf_nodes = 2**approx_depth - 1
        leaf_nodes = 2**approx_depth

        # Total parameters per tree
        params_per_tree = (
            non_leaf_nodes * 2 + leaf_nodes
        )  # split values + feature indices + leaf values
        return tree_count * params_per_tree
    

class GRANDETeacherModel(TeacherModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a GRANDE teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments. The `device` parameter is popped from kwargs and
            defaults to 'cpu' if not provided.
        """
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        self.cat_cols = kwargs.pop("cat_cols")
        super().__init__(**kwargs)

    def train(self, X, y):
        """
        Train the GRANDE teacher model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the GRANDE model's fit method.
        """
        config = self.config
        random_state = config["training"]["random_state"]

        # Determine categorical indices
        cat_indices = [i for i, is_cat in enumerate(self.cat_cols) if is_cat]

        training_params = {
            "epochs": 1000,
            "early_stopping_epochs": 25,
            "batch_size": 64,
            "cat_idx": cat_indices,
            "random_seed": random_state,
        }

        if self.task_type == "binary":
            training_params["objective"] = "binary"
        else:
            training_params["objective"] = "regression"

        # Instantiate the GRANDE teacher model.
        self.model = GRANDE(params=self.hyperparams, args=training_params)

        # Split the data into training and validation sets.
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )
        self.model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

    def predict(self, X):
        """
        Make predictions on the given input data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like
            The predicted target values.
        """
        if self.task_type == "binary":
            return self.model.predict(X)
        else:
            return self.model.predict(X).squeeze()

    def count_parameters(self):
        """
        Count the number of trainable parameters in the GRANDE teacher model based on the
        dense representation of a single tree and the number of trees in the ensemble.

        The number of parameters for a single tree is computed as:
            leaf parameters: 2^d
            split thresholds: (2^d - 1) * n
            feature selection (one-hot): (2^d - 1) * n
        so that:
            params_per_tree = 2^d + 2 * n * (2^d - 1)
        The total is then:
            total_params = n_estimators * params_per_tree
        """
        if not hasattr(self, "model"):
            raise ValueError("The model has not been trained/initialized yet.")

        # These attributes must be set when the GRANDE model is instantiated.
        # Adjust the attribute names if they are different in your GRANDE implementation.
        d = self.model.depth  # Depth of each decision tree.
        n = self.model.number_of_variables  # Number of features used in the tree.
        E = self.model.n_estimators  # Number of trees in the ensemble.

        # Calculate the number of parameters for one tree.
        params_per_tree = (2**d) + 2 * n * ((2**d) - 1)
        total_params = E * params_per_tree

        return total_params
    
class TabMTeacherModel(TeacherModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a TabM teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

        # Set up AMP if CUDA is available.
        if torch.cuda.is_available():
            self.amp_dtype = (
                torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
            )
            self.amp_enabled = True
        else:
            self.amp_dtype = None
            self.amp_enabled = False

    def train(self, X, y):
        """
        Train the TabM model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the TabM model's fit method.
        """
        # Get parameters
        n_blocks = self.hyperparams.get("n_blocks")
        d_block = self.hyperparams.get("d_block")
        dropout = self.hyperparams.get("dropout")
        lr = self.hyperparams.get("learning_rate")
        weight_decay = self.hyperparams.get("weight_decay")

        # Set model architecture
        arch_type = "tabm"
        bins = None
        n_features = X.shape[1]

        if self.task_type == "binary":
            n_classes = len(np.unique(y))
            self.n_classes = n_classes
            cat_idx = []  # Assumes all features are continuous

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=cat_idx,
                n_classes=n_classes,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=32,
                share_training_batches=True,
            ).to(self.device)

        else:
            if len(y.shape) > 1 and y.shape[1] > 1:
                # Multiclass logits
                n_outputs = y.shape[1]
            else:
                # Binary or single-output regression
                n_outputs = 1

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=[],  # Assumes all features are continuous
                n_classes=n_outputs,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=32,
                share_training_batches=True,
            ).to(self.device)

        self.optimizer = torch.optim.AdamW(
            make_parameter_groups(self.model), lr=lr, weight_decay=weight_decay
        )
        self.evaluation_mode = torch.no_grad if compile_model else torch.inference_mode

        # Prepare data
        X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
        if self.task_type == "binary":
            y_tensor = torch.tensor(y, dtype=torch.long, device=self.device)
        else:
            y_tensor = torch.tensor(y, dtype=torch.float32, device=self.device)
        n_samples = X_tensor.size(0)
        indices = torch.randperm(n_samples)
        train_end = int(0.8 * n_samples)
        train_idx, val_idx = indices[:train_end], indices[train_end:]
        self.data = {
            "train": {"x_cont": X_tensor[train_idx], "y": y_tensor[train_idx]},
            "val": {"x_cont": X_tensor[val_idx], "y": y_tensor[val_idx]},
        }
        Y_train = self.data["train"]["y"]

        # Training parameters
        n_epochs = 100000
        patience = 16
        batch_size = 256
        train_size = len(Y_train)
        best = {"val": -float("inf"), "epoch": -1}
        remaining_patience = patience

        # Training loop
        for epoch in range(n_epochs):
            batches = torch.randperm(train_size, device=self.device).split(batch_size)
            for batch_idx in batches:
                self.model.train()
                self.optimizer.zero_grad()
                loss = self.loss_fn(
                    self.apply_model("train", batch_idx), Y_train[batch_idx]
                )
                if self.amp_enabled and self.amp_dtype == torch.float16:
                    scaler = torch.cuda.amp.GradScaler()
                    scaler.scale(loss).backward()
                    scaler.step(self.optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    self.optimizer.step()
            val_score = self.evaluate_tabm("val")
            if val_score > best["val"]:
                best = {"val": val_score, "epoch": epoch}
                remaining_patience = patience
            else:
                remaining_patience -= 1
            if remaining_patience < 0:
                break

    def apply_model(self, part: str, idx: torch.Tensor) -> torch.Tensor:
        """
        Apply the model to a batch of data.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').
        idx : torch.Tensor
            The indices of the samples to use.

        Returns
        -------
        torch.Tensor
            The model output.
        """
        with torch.autocast(
            self.device.type, enabled=self.amp_enabled, dtype=self.amp_dtype
        ):
            x_cont = self.data[part]["x_cont"][idx]
            x_cat = self.data[part].get("x_cat")
            if self.task_type == "binary":
                return self.model(x_cont, x_cat).float()
            else:
                return (
                    self.model(x_cont, x_cat)
                    .squeeze(-1)  # Remove the last dimension for regression tasks
                    .float()
                )

    def loss_fn(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        """
        Compute the loss.

        Parameters
        ----------
        y_pred : torch.Tensor
            The predicted outputs.
        y_true : torch.Tensor
            The ground truth.

        Returns
        -------
        torch.Tensor
            The loss value.
        """
        if self.task_type == "binary":
            # Use the second-to-last dimension for averaging over k heads.
            k = y_pred.shape[-2]
            target = y_true.repeat_interleave(k)
            return F.cross_entropy(y_pred.flatten(0, 1), target)
        else:
            # TabM produces k predictions. Each of them must be trained separately.
            # For student model (regression on logits), y_pred.shape == (batch_size, k)
            k = y_pred.shape[-1 if len(y_pred.shape) <= 2 else -2]

            # Flatten the predictions and repeat the targets
            return F.mse_loss(
                y_pred.flatten(0, 1),
                (
                    y_true.repeat_interleave(k)
                    if self.model.share_training_batches
                    else y_true
                ),
            )

    def evaluate_tabm(self, part: str) -> float:
        """
        Evaluate the model on a data partition.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').

        Returns
        -------
        float
            The accuracy score.
        """
        self.model.eval()
        with self.evaluation_mode():
            eval_batch_size = 8096

            if self.task_type == "binary":
                indices = torch.arange(
                    len(self.data[part]["y"]), device=self.device
                ).split(eval_batch_size)
                y_pred = torch.cat(
                    [self.apply_model(part, idx) for idx in indices]
                ).cpu()
                y_pred = torch.softmax(y_pred, dim=-1).mean(1)
                y_true = self.data[part]["y"].cpu().numpy()
                return float(accuracy_score(y_true, y_pred.argmax(1)))
            else:
                y_pred = (
                    torch.cat(
                        [
                            self.apply_model(part, idx)
                            for idx in torch.arange(
                                len(self.data[part]["y"]), device=self.device
                            ).split(eval_batch_size)
                        ]
                    )
                    .cpu()
                    .numpy()
                )

                # Transform the predictions back to the original label space
                if hasattr(self, "regression_label_stats"):
                    y_pred = (
                        y_pred * self.regression_label_stats["std"]
                        + self.regression_label_stats["mean"]
                    )

                # Compute the mean of the k predictions
                y_pred = y_pred.mean(1)

                y_true = self.data[part]["y"].cpu().numpy()

                # For student model, we want to minimize MSE, so return negative MSE
                # The higher the value, the better (consistent with example.ipynb)
                score = -(mean_squared_error(y_true, y_pred) ** 0.5)
                return float(score)

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples,)
            The predicted class indices.
        """
        if self.task_type == "binary":
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                logits = self.model(X_tensor)
                # Since share_training_batches is True, average over the second dimension.
                probs = torch.softmax(logits, dim=-1).mean(1)
                return probs.cpu().numpy()
        else:
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                # TabM produces output with shape (batch_size, k) or (batch_size, k, n_outputs)
                logits = self.model(X_tensor)

                # For student model (regression), we need to average over the k predictions
                # Check the dimensions of logits and average appropriately
                if logits.ndim > 2:  # Case: (batch_size, k, n_outputs)
                    logits = logits.mean(1)  # Average over k dimension
                elif (
                    logits.shape[1] > 1
                    and hasattr(self.model, "k")
                    and self.model.k > 1
                ):
                    # Case: (batch_size, k) for single output regression
                    logits = logits.mean(
                        1, keepdim=True
                    )  # Average and keep output dimension

                return logits.cpu().numpy()

    def count_parameters(self):
        """
        Count the number of trainable parameters in the model.

        Returns
        -------
        int
            The total number of trainable parameters.
        """
        if not hasattr(self, "model"):
            raise ValueError("The model has not been trained yet")
        return sum(p.numel() for p in self.model.parameters() if p.requires_grad)    


class MLPTeacherModel(TeacherModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a TabM teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

        # Set up AMP if CUDA is available.
        if torch.cuda.is_available():
            self.amp_dtype = (
                torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
            )
            self.amp_enabled = True
        else:
            self.amp_dtype = None
            self.amp_enabled = False

    def train(self, X, y):
        """
        Train the TabM model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the TabM model's fit method.
        """
        # Get parameters
        n_blocks = self.hyperparams.get("n_blocks")
        d_block = self.hyperparams.get("d_block")
        dropout = self.hyperparams.get("dropout")
        lr = self.hyperparams.get("learning_rate")
        weight_decay = self.hyperparams.get("weight_decay")

        # Set model architecture
        arch_type = "plain"
        bins = None
        n_features = X.shape[1]

        if self.task_type == "binary":
            n_classes = len(np.unique(y))
            self.n_classes = n_classes
            cat_idx = []  # Assumes all features are continuous

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=cat_idx,
                n_classes=n_classes,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=None,
                share_training_batches=True,
            ).to(self.device)

        else:
            if len(y.shape) > 1 and y.shape[1] > 1:
                # Multiclass logits
                n_outputs = y.shape[1]
            else:
                # Binary or single-output regression
                n_outputs = 1

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=[],  # Assumes all features are continuous
                n_classes=n_outputs,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=None,
                share_training_batches=True,
            ).to(self.device)

        self.optimizer = torch.optim.AdamW(
            make_parameter_groups(self.model), lr=lr, weight_decay=weight_decay
        )
        self.evaluation_mode = torch.no_grad if compile_model else torch.inference_mode

        # Prepare data
        X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
        if self.task_type == "binary":
            y_tensor = torch.tensor(y, dtype=torch.long, device=self.device)
        else:
            y_tensor = torch.tensor(y, dtype=torch.float32, device=self.device)
        n_samples = X_tensor.size(0)
        indices = torch.randperm(n_samples)
        train_end = int(0.8 * n_samples)
        train_idx, val_idx = indices[:train_end], indices[train_end:]
        self.data = {
            "train": {"x_cont": X_tensor[train_idx], "y": y_tensor[train_idx]},
            "val": {"x_cont": X_tensor[val_idx], "y": y_tensor[val_idx]},
        }
        Y_train = self.data["train"]["y"]

        # Training parameters
        n_epochs = 100000
        patience = 16
        batch_size = 256
        train_size = len(Y_train)
        best = {"val": -float("inf"), "epoch": -1}
        remaining_patience = patience

        # Training loop
        for epoch in range(n_epochs):
            batches = torch.randperm(train_size, device=self.device).split(batch_size)
            for batch_idx in batches:
                self.model.train()
                self.optimizer.zero_grad()
                loss = self.loss_fn(
                    self.apply_model("train", batch_idx), Y_train[batch_idx]
                )
                if self.amp_enabled and self.amp_dtype == torch.float16:
                    scaler = torch.cuda.amp.GradScaler()
                    scaler.scale(loss).backward()
                    scaler.step(self.optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    self.optimizer.step()
            val_score = self.evaluate_tabm("val")
            if val_score > best["val"]:
                best = {"val": val_score, "epoch": epoch}
                remaining_patience = patience
            else:
                remaining_patience -= 1
            if remaining_patience < 0:
                break

    def apply_model(self, part: str, idx: torch.Tensor) -> torch.Tensor:
        """
        Apply the model to a batch of data.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').
        idx : torch.Tensor
            The indices of the samples to use.

        Returns
        -------
        torch.Tensor
            The model output.
        """
        with torch.autocast(
            self.device.type, enabled=self.amp_enabled, dtype=self.amp_dtype
        ):
            x_cont = self.data[part]["x_cont"][idx]
            x_cat = self.data[part].get("x_cat")
            if self.task_type == "binary":
                return self.model(x_cont, x_cat).float()
            else:
                return (
                    self.model(x_cont, x_cat)
                    .squeeze(-1)  # Remove the last dimension for regression tasks
                    .float()
                )

    def loss_fn(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        """
        Compute the loss.

        Parameters
        ----------
        y_pred : torch.Tensor
            The predicted outputs.
        y_true : torch.Tensor
            The ground truth.

        Returns
        -------
        torch.Tensor
            The loss value.
        """
        if self.task_type == "binary":
            # Use the second-to-last dimension for averaging over k heads.
            k = y_pred.shape[-2]
            target = y_true.repeat_interleave(k)
            return F.cross_entropy(y_pred.flatten(0, 1), target)
        else:
            # TabM produces k predictions. Each of them must be trained separately.
            # For student model (regression on logits), y_pred.shape == (batch_size, k)
            k = y_pred.shape[-1 if len(y_pred.shape) <= 2 else -2]

            # Flatten the predictions and repeat the targets
            return F.mse_loss(
                y_pred.flatten(0, 1),
                (
                    y_true.repeat_interleave(k)
                    if self.model.share_training_batches
                    else y_true
                ),
            )

    def evaluate_tabm(self, part: str) -> float:
        """
        Evaluate the model on a data partition.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').

        Returns
        -------
        float
            The accuracy score.
        """
        self.model.eval()
        with self.evaluation_mode():
            eval_batch_size = 8096

            if self.task_type == "binary":
                indices = torch.arange(
                    len(self.data[part]["y"]), device=self.device
                ).split(eval_batch_size)
                y_pred = torch.cat(
                    [self.apply_model(part, idx) for idx in indices]
                ).cpu()
                y_pred = torch.softmax(y_pred, dim=-1).mean(1)
                y_true = self.data[part]["y"].cpu().numpy()
                return float(accuracy_score(y_true, y_pred.argmax(1)))
            else:
                y_pred = (
                    torch.cat(
                        [
                            self.apply_model(part, idx)
                            for idx in torch.arange(
                                len(self.data[part]["y"]), device=self.device
                            ).split(eval_batch_size)
                        ]
                    )
                    .cpu()
                    .numpy()
                )

                # Transform the predictions back to the original label space
                if hasattr(self, "regression_label_stats"):
                    y_pred = (
                        y_pred * self.regression_label_stats["std"]
                        + self.regression_label_stats["mean"]
                    )

                # Compute the mean of the k predictions
                y_pred = y_pred.mean(1)

                y_true = self.data[part]["y"].cpu().numpy()

                # For student model, we want to minimize MSE, so return negative MSE
                # The higher the value, the better (consistent with example.ipynb)
                score = -(mean_squared_error(y_true, y_pred) ** 0.5)
                return float(score)

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples,)
            The predicted class indices.
        """
        if self.task_type == "binary":
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                logits = self.model(X_tensor)
                # Since share_training_batches is True, average over the second dimension.
                probs = torch.softmax(logits, dim=-1).mean(1)
                return probs.cpu().numpy()
        else:
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                # TabM produces output with shape (batch_size, k) or (batch_size, k, n_outputs)
                logits = self.model(X_tensor)

                # For student model (regression), we need to average over the k predictions
                # Check the dimensions of logits and average appropriately
                if logits.ndim > 2:  # Case: (batch_size, k, n_outputs)
                    logits = logits.mean(1)  # Average over k dimension
                elif (
                    logits.shape[1] > 1
                    and hasattr(self.model, "k")
                    and self.model.k > 1
                ):
                    # Case: (batch_size, k) for single output regression
                    logits = logits.mean(
                        1, keepdim=True
                    )  # Average and keep output dimension

                return logits.cpu().numpy()

    def count_parameters(self):
        """
        Count the number of trainable parameters in the model.

        Returns
        -------
        int
            The total number of trainable parameters.
        """
        if not hasattr(self, "model"):
            raise ValueError("The model has not been trained yet")
        return sum(p.numel() for p in self.model.parameters() if p.requires_grad)


class RandomForestTeacherModel(TeacherModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a RandomForest teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

    def train(self, X, y):
        """
        Train the RandomForest teacher model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters (not used for RandomForest).
        """
        config = self.config
        random_state = config["training"]["random_state"]

        if self.task_type == "binary":
            # Initialize the RandomForest model with the provided hyperparameters
            self.model = RandomForestClassifier(
                random_state=random_state,
                n_jobs=-1,  # Use all available CPU cores
                **self.hyperparams,
            )
        else:
            self.model = RandomForestRegressor(
                random_state=random_state,
                n_jobs=-1,  # Use all available CPU cores
                **self.hyperparams,
            )

        # Train the model
        self.model.fit(X, y)

    def predict(self, X):
        """
        Make probability predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples, n_classes)
            The predicted probabilities for each class.
        """
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self):
        """
        Count the number of parameters in the trained RandomForest model.

        For each tree, we count:
        - The feature indices at each node (1 parameter per node except leaf nodes)
        - The thresholds at each node (1 parameter per node except leaf nodes)
        - The values at each leaf node (1 parameter per leaf node)

        Returns
        -------
        int
            The number of parameters in the model.
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("The model has not been trained yet.")

        n_params = 0
        for tree in self.model.estimators_:
            tree = tree.tree_
            # Number of nodes in the tree
            n_nodes = tree.node_count
            # Number of leaf nodes
            n_leaves = tree.n_leaves
            # Number of internal nodes
            n_internal = n_nodes - n_leaves
            # Each internal node has a feature index and a threshold
            n_params += 2 * n_internal

            # Each leaf node has value(s) - depends on task type
            if self.task_type == "binary":
                # For classification, each leaf stores class distribution
                n_params += n_leaves * self.model.n_classes_
            else:
                # For regression, each leaf stores a single value
                n_params += n_leaves

        return n_params
    
def get_teacher_model(config: dict, task_type: Literal["binary", "regression"], device, hyperparams, cat_cols=None) -> TeacherModelBase:
    """
    Returns an instance of the teacher model based on the configuration.

    Parameters
    ----------
    config : dict
        Configuration dictionary containing model and dataset details.
    device : torch.device
        The device to use for training (CPU or GPU).

    Returns
    -------
    TeacherModelBase
        An instance of the teacher model.
    """
    model_type = config["model"]["teacher_model"]
    if model_type == "tabpfn":
        return TabPFNTeacherModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "catboost":
        return CatBoostTeacherModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "grande":
        return GRANDETeacherModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams, cat_cols=cat_cols)
    elif model_type == "tabm":
        return TabMTeacherModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "mlp":
        return MLPTeacherModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "random_forest":
        return RandomForestTeacherModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    else:
        raise ValueError(f"Unknown teacher type: {config['model']['teacher_model']}")

## Student

In [7]:
class StudentModelBase(ABC):
    def __init__(self, **kwargs):
        self.params = kwargs

    @abstractmethod
    def train(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Trains the model on the given data.

        Parameters:
            X (np.ndarray): The input features with shape (n_samples, n_features).
            y (np.ndarray): The target labels with shape (n_samples,).

        Returns:
            None
        """
        pass

    @abstractmethod
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Makes predictions on the provided data.

        Parameters:
            X (np.ndarray): The input features with shape (n_samples, n_features).

        Returns:
            np.ndarray: An array of predicted probabilities with shape (n_samples, n_classes).
                        (If the model returns logits, these should be post-processed to probabilities.)
        """
        pass

    def evaluate(self, y_pred: np.ndarray, y_true: np.ndarray, y_teacher_true: np.ndarray, original_task_type: Literal["binary", "regression"]) -> Dict[str, float]:
        """
        Evaluates the model's predictions against the true labels.

        Parameters:
            y_pred (np.ndarray): The predicted probabilities or values.
            y_true (np.ndarray): The true labels.

        Returns:
            Dict[str, float]: A dictionary containing evaluation metrics.
        """     
        if original_task_type == "binary":
            if self.task_type == "binary":
                metrics = evaluate_classification(y_pred, y_true)
                fidelity_metrics = evaluate_fidelity_classification(y_pred, y_teacher_true)
                metrics.update(fidelity_metrics)
                metrics['parameters'] = self.count_parameters()
                return metrics
            
            elif self.task_type == "regression":
                # Convert logits to probabilities
                y_prob = 1 / (1 + np.exp(-y_pred))  # Sigmoid function
                y_prob = np.column_stack([1 - y_prob, y_prob])  # Create 2D array with probs for both classes

                y_teacher_prob = 1 / (1 + np.exp(-y_teacher_true))
                y_teacher_prob = np.column_stack([1 - y_teacher_prob, y_teacher_prob])
                
                metrics = evaluate_classification(y_prob, y_true)
                class_fidelity_metrics = evaluate_fidelity_classification(y_prob, y_teacher_prob)
                metrics.update(class_fidelity_metrics)
                reg_fidelity_metrics = evaluate_fidelity_regression(y_pred, y_teacher_true)
                metrics.update(reg_fidelity_metrics)
                metrics['parameters'] = self.count_parameters()
                return metrics
            
            else:
                raise ValueError(f"Unknown task type: {self.task_type}. Must be 'binary' or 'regression'.")
        
        elif original_task_type == "regression":
            metrics = evaluate_regression(y_pred, y_true)
            fidelity_metrics = evaluate_fidelity_regression(y_pred, y_teacher_true)
            metrics.update(fidelity_metrics)
            metrics['parameters'] = self.count_parameters()
            return metrics
        
        else:
            raise ValueError(f"Unknown task type: {self.task_type}. Must be 'binary' or 'regression'.")

    @abstractmethod
    def count_parameters(self) -> int:
        """
        Counts the number of trainable parameters in the model.

        Returns:
            int: The total number of trainable parameters.
        """
        pass


class CatBoostStudentModel(StudentModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a CatBoost teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

    def train(self, X, y):
        """
        Train the CatBoost model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the CatBoost model's fit method.
        """
        config = self.config
        random_state = config["training"]["random_state"]

        # Initialize the CatBoost model
        if self.task_type == "binary":
            self.model = CatBoostClassifier(
                random_seed=random_state,
                thread_count=-1,  # Use all available CPU cores
                logging_level="Silent",  # Suppress CatBoost output
                task_type=("GPU" if str(self.device).lower() == "cuda" else "CPU"),
                **self.hyperparams,  # Include the sampled hyperparameters
            )
        else:
            self.model = CatBoostRegressor(
                random_seed=random_state,
                thread_count=-1,  # Use all available CPU cores
                logging_level="Silent",  # Suppress CatBoost output
                task_type=("GPU" if str(self.device).lower() == "cuda" else "CPU"),
                **self.hyperparams,  # Include the sampled hyperparameters
            )

        # Split data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )
        # Train the model
        self.model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=20, 
        )

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples, n_classes)
            The predicted probabilities for each class.
        """
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self):
        """
        Count the number of parameters in the trained CatBoost model.

        Returns
        -------
        int
            The number of parameters in the model.
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("The model has not been trained yet.")

        # Get the number of trees in the model
        tree_count = self.model.tree_count_

        # For each tree, count:
        # - split values (one per non-leaf node)
        # - leaf values (one per leaf node)
        # - feature indices (one per non-leaf node)
        # For a binary tree with depth d, there are 2^d - 1 non-leaf nodes and 2^d leaf nodes
        # This is a simplified approximation
        approx_depth = self.hyperparams.get("max_depth", 6)  # Default depth in CatBoost is 6
        non_leaf_nodes = 2**approx_depth - 1
        leaf_nodes = 2**approx_depth

        # Total parameters per tree
        params_per_tree = (
            non_leaf_nodes * 2 + leaf_nodes
        )  # split values + feature indices + leaf values
        return tree_count * params_per_tree
    

class GRANDEStudentModel(StudentModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a GRANDE teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments. The `device` parameter is popped from kwargs and
            defaults to 'cpu' if not provided.
        """
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        self.cat_cols = kwargs.pop("cat_cols")
        super().__init__(**kwargs)

    def train(self, X, y):
        """
        Train the GRANDE teacher model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the GRANDE model's fit method.
        """
        config = self.config
        random_state = config["training"]["random_state"]

        # Determine categorical indices
        cat_indices = [i for i, is_cat in enumerate(self.cat_cols) if is_cat]

        training_params = {
            "epochs": 1000,
            "early_stopping_epochs": 25,
            "batch_size": 64,
            "cat_idx": cat_indices,
            "random_seed": random_state,
        }

        if self.task_type == "binary":
            training_params["objective"] = "binary"
        else:
            training_params["objective"] = "regression"

        # Instantiate the GRANDE teacher model.
        self.model = GRANDE(params=self.hyperparams, args=training_params)

        # Split the data into training and validation sets.
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )
        self.model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

    def predict(self, X):
        """
        Make predictions on the given input data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like
            The predicted target values.
        """
        if self.task_type == "binary":
            return self.model.predict(X)
        else:
            return self.model.predict(X).squeeze()

    def count_parameters(self):
        """
        Count the number of trainable parameters in the GRANDE teacher model based on the
        dense representation of a single tree and the number of trees in the ensemble.

        The number of parameters for a single tree is computed as:
            leaf parameters: 2^d
            split thresholds: (2^d - 1) * n
            feature selection (one-hot): (2^d - 1) * n
        so that:
            params_per_tree = 2^d + 2 * n * (2^d - 1)
        The total is then:
            total_params = n_estimators * params_per_tree
        """
        if not hasattr(self, "model"):
            raise ValueError("The model has not been trained/initialized yet.")

        # These attributes must be set when the GRANDE model is instantiated.
        # Adjust the attribute names if they are different in your GRANDE implementation.
        d = self.model.depth  # Depth of each decision tree.
        n = self.model.number_of_variables  # Number of features used in the tree.
        E = self.model.n_estimators  # Number of trees in the ensemble.

        # Calculate the number of parameters for one tree.
        params_per_tree = (2**d) + 2 * n * ((2**d) - 1)
        total_params = E * params_per_tree

        return total_params
    
class TabMStudentModel(StudentModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a TabM teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

        # Set up AMP if CUDA is available.
        if torch.cuda.is_available():
            self.amp_dtype = (
                torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
            )
            self.amp_enabled = True
        else:
            self.amp_dtype = None
            self.amp_enabled = False

    def train(self, X, y):
        """
        Train the TabM model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the TabM model's fit method.
        """
        # Get parameters
        n_blocks = self.hyperparams.get("n_blocks")
        d_block = self.hyperparams.get("d_block")
        dropout = self.hyperparams.get("dropout")
        lr = self.hyperparams.get("learning_rate")
        weight_decay = self.hyperparams.get("weight_decay")

        # Set model architecture
        arch_type = "tabm"
        bins = None
        n_features = X.shape[1]

        if self.task_type == "binary":
            n_classes = len(np.unique(y))
            self.n_classes = n_classes
            cat_idx = []  # Assumes all features are continuous

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=cat_idx,
                n_classes=n_classes,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=32,
                share_training_batches=True,
            ).to(self.device)

        else:
            if len(y.shape) > 1 and y.shape[1] > 1:
                # Multiclass logits
                n_outputs = y.shape[1]
            else:
                # Binary or single-output regression
                n_outputs = 1

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=[],  # Assumes all features are continuous
                n_classes=n_outputs,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=32,
                share_training_batches=True,
            ).to(self.device)

        self.optimizer = torch.optim.AdamW(
            make_parameter_groups(self.model), lr=lr, weight_decay=weight_decay
        )
        self.evaluation_mode = torch.no_grad if compile_model else torch.inference_mode

        # Prepare data
        X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
        if self.task_type == "binary":
            y_tensor = torch.tensor(y, dtype=torch.long, device=self.device)
        else:
            y_tensor = torch.tensor(y, dtype=torch.float32, device=self.device)
        n_samples = X_tensor.size(0)
        indices = torch.randperm(n_samples)
        train_end = int(0.8 * n_samples)
        train_idx, val_idx = indices[:train_end], indices[train_end:]
        self.data = {
            "train": {"x_cont": X_tensor[train_idx], "y": y_tensor[train_idx]},
            "val": {"x_cont": X_tensor[val_idx], "y": y_tensor[val_idx]},
        }
        Y_train = self.data["train"]["y"]

        # Training parameters
        n_epochs = 100000
        patience = 16
        batch_size = 256
        train_size = len(Y_train)
        best = {"val": -float("inf"), "epoch": -1}
        remaining_patience = patience

        # Training loop
        for epoch in range(n_epochs):
            batches = torch.randperm(train_size, device=self.device).split(batch_size)
            for batch_idx in batches:
                self.model.train()
                self.optimizer.zero_grad()
                loss = self.loss_fn(
                    self.apply_model("train", batch_idx), Y_train[batch_idx]
                )
                if self.amp_enabled and self.amp_dtype == torch.float16:
                    scaler = torch.cuda.amp.GradScaler()
                    scaler.scale(loss).backward()
                    scaler.step(self.optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    self.optimizer.step()
            val_score = self.evaluate_tabm("val")
            if val_score > best["val"]:
                best = {"val": val_score, "epoch": epoch}
                remaining_patience = patience
            else:
                remaining_patience -= 1
            if remaining_patience < 0:
                break

    def apply_model(self, part: str, idx: torch.Tensor) -> torch.Tensor:
        """
        Apply the model to a batch of data.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').
        idx : torch.Tensor
            The indices of the samples to use.

        Returns
        -------
        torch.Tensor
            The model output.
        """
        with torch.autocast(
            self.device.type, enabled=self.amp_enabled, dtype=self.amp_dtype
        ):
            x_cont = self.data[part]["x_cont"][idx]
            x_cat = self.data[part].get("x_cat")
            if self.task_type == "binary":
                return self.model(x_cont, x_cat).float()
            else:
                return (
                    self.model(x_cont, x_cat)
                    .squeeze(-1)  # Remove the last dimension for regression tasks
                    .float()
                )

    def loss_fn(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        """
        Compute the loss.

        Parameters
        ----------
        y_pred : torch.Tensor
            The predicted outputs.
        y_true : torch.Tensor
            The ground truth.

        Returns
        -------
        torch.Tensor
            The loss value.
        """
        if self.task_type == "binary":
            # Use the second-to-last dimension for averaging over k heads.
            k = y_pred.shape[-2]
            target = y_true.repeat_interleave(k)
            return F.cross_entropy(y_pred.flatten(0, 1), target)
        else:
            # TabM produces k predictions. Each of them must be trained separately.
            # For student model (regression on logits), y_pred.shape == (batch_size, k)
            k = y_pred.shape[-1 if len(y_pred.shape) <= 2 else -2]

            # Flatten the predictions and repeat the targets
            return F.mse_loss(
                y_pred.flatten(0, 1),
                (
                    y_true.repeat_interleave(k)
                    if self.model.share_training_batches
                    else y_true
                ),
            )

    def evaluate_tabm(self, part: str) -> float:
        """
        Evaluate the model on a data partition.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').

        Returns
        -------
        float
            The accuracy score.
        """
        self.model.eval()
        with self.evaluation_mode():
            eval_batch_size = 8096

            if self.task_type == "binary":
                indices = torch.arange(
                    len(self.data[part]["y"]), device=self.device
                ).split(eval_batch_size)
                y_pred = torch.cat(
                    [self.apply_model(part, idx) for idx in indices]
                ).cpu()
                y_pred = torch.softmax(y_pred, dim=-1).mean(1)
                y_true = self.data[part]["y"].cpu().numpy()
                return float(accuracy_score(y_true, y_pred.argmax(1)))
            else:
                y_pred = (
                    torch.cat(
                        [
                            self.apply_model(part, idx)
                            for idx in torch.arange(
                                len(self.data[part]["y"]), device=self.device
                            ).split(eval_batch_size)
                        ]
                    )
                    .cpu()
                    .numpy()
                )

                # Transform the predictions back to the original label space
                if hasattr(self, "regression_label_stats"):
                    y_pred = (
                        y_pred * self.regression_label_stats["std"]
                        + self.regression_label_stats["mean"]
                    )

                # Compute the mean of the k predictions
                y_pred = y_pred.mean(1)

                y_true = self.data[part]["y"].cpu().numpy()

                # For student model, we want to minimize MSE, so return negative MSE
                # The higher the value, the better (consistent with example.ipynb)
                score = -(mean_squared_error(y_true, y_pred) ** 0.5)
                return float(score)

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples,)
            The predicted class indices.
        """
        if self.task_type == "binary":
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                logits = self.model(X_tensor)
                # Since share_training_batches is True, average over the second dimension.
                probs = torch.softmax(logits, dim=-1).mean(1)
                return probs.cpu().numpy()
        else:
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                # TabM produces output with shape (batch_size, k) or (batch_size, k, n_outputs)
                logits = self.model(X_tensor)

                # For student model (regression), we need to average over the k predictions
                # Check the dimensions of logits and average appropriately
                if logits.ndim > 2:  # Case: (batch_size, k, n_outputs)
                    logits = logits.mean(1)  # Average over k dimension
                elif (
                    logits.shape[1] > 1
                    and hasattr(self.model, "k")
                    and self.model.k > 1
                ):
                    # Case: (batch_size, k) for single output regression
                    logits = logits.mean(
                        1, keepdim=True
                    )  # Average and keep output dimension

                return logits.cpu().numpy().squeeze(-1)  # Squeeze to remove last dimension if needed

    def count_parameters(self):
        """
        Count the number of trainable parameters in the model.

        Returns
        -------
        int
            The total number of trainable parameters.
        """
        if not hasattr(self, "model"):
            raise ValueError("The model has not been trained yet")
        return sum(p.numel() for p in self.model.parameters() if p.requires_grad)    


class MLPStudentModel(StudentModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a TabM teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

        # Set up AMP if CUDA is available.
        if torch.cuda.is_available():
            self.amp_dtype = (
                torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
            )
            self.amp_enabled = True
        else:
            self.amp_dtype = None
            self.amp_enabled = False

    def train(self, X, y):
        """
        Train the TabM model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the TabM model's fit method.
        """
        # Get parameters
        n_blocks = self.hyperparams.get("n_blocks")
        d_block = self.hyperparams.get("d_block")
        dropout = self.hyperparams.get("dropout")
        lr = self.hyperparams.get("learning_rate")
        weight_decay = self.hyperparams.get("weight_decay")

        # Set model architecture
        arch_type = "plain"
        bins = None
        n_features = X.shape[1]

        if self.task_type == "binary":
            n_classes = len(np.unique(y))
            self.n_classes = n_classes
            cat_idx = []  # Assumes all features are continuous

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=cat_idx,
                n_classes=n_classes,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=None,
                share_training_batches=True,
            ).to(self.device)

        else:
            if len(y.shape) > 1 and y.shape[1] > 1:
                # Multiclass logits
                n_outputs = y.shape[1]
            else:
                # Binary or single-output regression
                n_outputs = 1

            # Initialize model
            compile_model = False
            self.model = Model(
                n_num_features=n_features,
                cat_cardinalities=[],  # Assumes all features are continuous
                n_classes=n_outputs,
                backbone={
                    "type": "MLP",
                    "n_blocks": n_blocks,
                    "d_block": d_block,
                    "dropout": dropout,
                },
                bins=bins,
                num_embeddings=(
                    None
                    if bins is None
                    else {
                        "type": "PiecewiseLinearEmbeddings",
                        "d_embedding": 16,
                        "activation": False,
                        "version": "B",
                    }
                ),
                arch_type=arch_type,
                k=None,
                share_training_batches=True,
            ).to(self.device)

        self.optimizer = torch.optim.AdamW(
            make_parameter_groups(self.model), lr=lr, weight_decay=weight_decay
        )
        self.evaluation_mode = torch.no_grad if compile_model else torch.inference_mode

        # Prepare data
        X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
        if self.task_type == "binary":
            y_tensor = torch.tensor(y, dtype=torch.long, device=self.device)
        else:
            y_tensor = torch.tensor(y, dtype=torch.float32, device=self.device)
        n_samples = X_tensor.size(0)
        indices = torch.randperm(n_samples)
        train_end = int(0.8 * n_samples)
        train_idx, val_idx = indices[:train_end], indices[train_end:]
        self.data = {
            "train": {"x_cont": X_tensor[train_idx], "y": y_tensor[train_idx]},
            "val": {"x_cont": X_tensor[val_idx], "y": y_tensor[val_idx]},
        }
        Y_train = self.data["train"]["y"]

        # Training parameters
        n_epochs = 100000
        patience = 16
        batch_size = 256
        train_size = len(Y_train)
        best = {"val": -float("inf"), "epoch": -1}
        remaining_patience = patience

        # Training loop
        for epoch in range(n_epochs):
            batches = torch.randperm(train_size, device=self.device).split(batch_size)
            for batch_idx in batches:
                self.model.train()
                self.optimizer.zero_grad()
                loss = self.loss_fn(
                    self.apply_model("train", batch_idx), Y_train[batch_idx]
                )
                if self.amp_enabled and self.amp_dtype == torch.float16:
                    scaler = torch.cuda.amp.GradScaler()
                    scaler.scale(loss).backward()
                    scaler.step(self.optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    self.optimizer.step()
            val_score = self.evaluate_tabm("val")
            if val_score > best["val"]:
                best = {"val": val_score, "epoch": epoch}
                remaining_patience = patience
            else:
                remaining_patience -= 1
            if remaining_patience < 0:
                break

    def apply_model(self, part: str, idx: torch.Tensor) -> torch.Tensor:
        """
        Apply the model to a batch of data.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').
        idx : torch.Tensor
            The indices of the samples to use.

        Returns
        -------
        torch.Tensor
            The model output.
        """
        with torch.autocast(
            self.device.type, enabled=self.amp_enabled, dtype=self.amp_dtype
        ):
            x_cont = self.data[part]["x_cont"][idx]
            x_cat = self.data[part].get("x_cat")
            if self.task_type == "binary":
                return self.model(x_cont, x_cat).float()
            else:
                return (
                    self.model(x_cont, x_cat)
                    .squeeze(-1)  # Remove the last dimension for regression tasks
                    .float()
                )

    def loss_fn(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        """
        Compute the loss.

        Parameters
        ----------
        y_pred : torch.Tensor
            The predicted outputs.
        y_true : torch.Tensor
            The ground truth.

        Returns
        -------
        torch.Tensor
            The loss value.
        """
        if self.task_type == "binary":
            # Use the second-to-last dimension for averaging over k heads.
            k = y_pred.shape[-2]
            target = y_true.repeat_interleave(k)
            return F.cross_entropy(y_pred.flatten(0, 1), target)
        else:
            # TabM produces k predictions. Each of them must be trained separately.
            # For student model (regression on logits), y_pred.shape == (batch_size, k)
            k = y_pred.shape[-1 if len(y_pred.shape) <= 2 else -2]

            # Flatten the predictions and repeat the targets
            return F.mse_loss(
                y_pred.flatten(0, 1),
                (
                    y_true.repeat_interleave(k)
                    if self.model.share_training_batches
                    else y_true
                ),
            )

    def evaluate_tabm(self, part: str) -> float:
        """
        Evaluate the model on a data partition.

        Parameters
        ----------
        part : str
            The data partition to use ('train' or 'val').

        Returns
        -------
        float
            The accuracy score.
        """
        self.model.eval()
        with self.evaluation_mode():
            eval_batch_size = 8096

            if self.task_type == "binary":
                indices = torch.arange(
                    len(self.data[part]["y"]), device=self.device
                ).split(eval_batch_size)
                y_pred = torch.cat(
                    [self.apply_model(part, idx) for idx in indices]
                ).cpu()
                y_pred = torch.softmax(y_pred, dim=-1).mean(1)
                y_true = self.data[part]["y"].cpu().numpy()
                return float(accuracy_score(y_true, y_pred.argmax(1)))
            else:
                y_pred = (
                    torch.cat(
                        [
                            self.apply_model(part, idx)
                            for idx in torch.arange(
                                len(self.data[part]["y"]), device=self.device
                            ).split(eval_batch_size)
                        ]
                    )
                    .cpu()
                    .numpy()
                )

                # Transform the predictions back to the original label space
                if hasattr(self, "regression_label_stats"):
                    y_pred = (
                        y_pred * self.regression_label_stats["std"]
                        + self.regression_label_stats["mean"]
                    )

                # Compute the mean of the k predictions
                y_pred = y_pred.mean(1)

                y_true = self.data[part]["y"].cpu().numpy()

                # For student model, we want to minimize MSE, so return negative MSE
                # The higher the value, the better (consistent with example.ipynb)
                score = -(mean_squared_error(y_true, y_pred) ** 0.5)
                return float(score)

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples,)
            The predicted class indices.
        """
        if self.task_type == "binary":
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                logits = self.model(X_tensor)
                # Since share_training_batches is True, average over the second dimension.
                probs = torch.softmax(logits, dim=-1).mean(1)
                return probs.cpu().numpy()
        else:
            self.model.eval()
            X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                # TabM produces output with shape (batch_size, k) or (batch_size, k, n_outputs)
                logits = self.model(X_tensor)

                # For student model (regression), we need to average over the k predictions
                # Check the dimensions of logits and average appropriately
                if logits.ndim > 2:  # Case: (batch_size, k, n_outputs)
                    logits = logits.mean(1)  # Average over k dimension
                elif (
                    logits.shape[1] > 1
                    and hasattr(self.model, "k")
                    and self.model.k > 1
                ):
                    # Case: (batch_size, k) for single output regression
                    logits = logits.mean(
                        1, keepdim=True
                    )  # Average and keep output dimension

                return logits.cpu().numpy().squeeze(-1)  # Squeeze to remove last dimension if needed

    def count_parameters(self):
        """
        Count the number of trainable parameters in the model.

        Returns
        -------
        int
            The total number of trainable parameters.
        """
        if not hasattr(self, "model"):
            raise ValueError("The model has not been trained yet")
        return sum(p.numel() for p in self.model.parameters() if p.requires_grad)



class RandomForestStudentModel(StudentModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a RandomForest teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        self.hyperparams = kwargs.pop("hyperparams")
        super().__init__(**kwargs)

    def train(self, X, y):
        """
        Train the RandomForest teacher model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters (not used for RandomForest).
        """
        config = self.config
        random_state = config["training"]["random_state"]

        if self.task_type == "binary":
            # Initialize the RandomForest model with the provided hyperparameters
            self.model = RandomForestClassifier(
                random_state=random_state,
                n_jobs=-1,  # Use all available CPU cores
                **self.hyperparams,
            )
        else:
            self.model = RandomForestRegressor(
                random_state=random_state,
                n_jobs=-1,  # Use all available CPU cores
                **self.hyperparams,
            )

        # Train the model
        self.model.fit(X, y)

    def predict(self, X):
        """
        Make probability predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples, n_classes)
            The predicted probabilities for each class.
        """
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self):
        """
        Count the number of parameters in the trained RandomForest model.

        For each tree, we count:
        - The feature indices at each node (1 parameter per node except leaf nodes)
        - The thresholds at each node (1 parameter per node except leaf nodes)
        - The values at each leaf node (1 parameter per leaf node)

        Returns
        -------
        int
            The number of parameters in the model.
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("The model has not been trained yet.")

        n_params = 0
        for tree in self.model.estimators_:
            tree = tree.tree_
            # Number of nodes in the tree
            n_nodes = tree.node_count
            # Number of leaf nodes
            n_leaves = tree.n_leaves
            # Number of internal nodes
            n_internal = n_nodes - n_leaves
            # Each internal node has a feature index and a threshold
            n_params += 2 * n_internal

            # Each leaf node has value(s) - depends on task type
            if self.task_type == "binary":
                # For classification, each leaf stores class distribution
                n_params += n_leaves * self.model.n_classes_
            else:
                # For regression, each leaf stores a single value
                n_params += n_leaves

        return n_params
    
def get_student_model(config: dict, task_type: Literal["binary", "regression"], device, hyperparams, cat_cols=None) -> StudentModelBase:
    """
    Returns an instance of the teacher model based on the configuration.

    Parameters
    ----------
    config : dict
        Configuration dictionary containing model and dataset details.
    device : torch.device
        The device to use for training (CPU or GPU).

    Returns
    -------
    TeacherModelBase
        An instance of the teacher model.
    """
    model_type = config["model"]["student_model"]
    if model_type == "catboost":
        return CatBoostStudentModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "grande":
        return GRANDEStudentModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams, cat_cols=cat_cols)
    elif model_type == "tabm":
        return TabMStudentModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "mlp":
        return MLPStudentModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    elif model_type == "random_forest":
        return RandomForestStudentModel(config=config, task_type=task_type, device=device, hyperparams=hyperparams)
    else:
        raise ValueError(f"Unknown teacher type: {config['model']['student_model']}. ")

# HPO

In [8]:
def suggest_hyperparameters(trial: optuna.Trial, model_type: str, task_type: str, use_hpo: bool) -> dict:
    """
    Suggest hyperparameters for the given model type using Optuna.
    
    Parameters
    ----------
    trial : optuna.Trial
        Optuna trial object for suggesting hyperparameters.
    model_type : str
        Type of model ('tabpfn', 'catboost', etc.).
    task_type : str
        Type of task ('binary', 'regression').
        
    Returns
    -------
    dict
        Dictionary of suggested hyperparameters.
    """
    hyperparameter = {}
    
    if model_type == "catboost":
        if use_hpo:
            hyperparameter = {
                "iterations": trial.suggest_int("iterations", 512, 4096),
                "max_depth": trial.suggest_int("max_depth", 2, 12),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.5, 30),
                "boosting_type": "Plain",
            }
        else:
            hyperparameter = {
                "iterations": 1000,
                "max_depth": 6,
                "l2_leaf_reg": 3,
                "boosting_type": "Plain",
            }
        if task_type == "binary":
            hyperparameter["loss_function"] = "Logloss"
        else:
            hyperparameter["loss_function"] = "RMSE"
    
    elif model_type == "grande":
        if use_hpo:
            hyperparameter = {
                "depth": trial.suggest_int("depth", 3, 7),
                "n_estimators": trial.suggest_int("n_estimators", 512, 4096),
                "learning_rate_weights": trial.suggest_float("learning_rate_weights", 0.0001, 0.05, log=True),
                "learning_rate_index": trial.suggest_float("learning_rate_index", 0.001, 0.2, log=True),
                "learning_rate_values": trial.suggest_float("learning_rate_values", 0.001, 0.2, log=True),
                "learning_rate_leaf": trial.suggest_float("learning_rate_leaf", 0.001, 0.2, log=True),
                "optimizer": "adam",
                "cosine_decay_steps": trial.suggest_categorical("cosine_decay_steps", [0.0, 0.1, 1.0, 100.0, 1000.0]),
                "dropout": trial.suggest_float("dropout", 0.0, 0.75),
                "selected_variables": trial.suggest_float("selected_variables", 0.0, 1.0),
                "data_subset_fraction": trial.suggest_float("data_subset_fraction", 0.1, 1.0),
                "focal_loss": False,
                "temperature": 0.0,
                "from_logits": True,
                "use_class_weights": True,
            }
        else:
            hyperparameter = {
                "depth": 5,
                "n_estimators": 2048,
                "learning_rate_weights": 0.005,
                "learning_rate_index": 0.01,
                "learning_rate_values": 0.01,
                "learning_rate_leaf": 0.01,
                "optimizer": "adam",
                "cosine_decay_steps": 0,
                "dropout": 0.0,
                "selected_variables": 0.8,
                "data_subset_fraction": 1.0,
                "focal_loss": False,
                "temperature": 0.0,
                "from_logits": True,
                "use_class_weights": True,
            }
        if task_type == "binary":
            hyperparameter["loss"] = "crossentropy"
        else:
            hyperparameter["loss"] = "mse"

    elif model_type == "tabm":
        if use_hpo:
            hyperparameter = {
                "n_blocks": trial.suggest_int("n_blocks", 1, 5),
                "d_block": trial.suggest_int("d_block", 64, 1024),
                "dropout": trial.suggest_float("dropout", 0.0, 0.5),
                "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.005, log=True),
                "weight_decay": trial.suggest_float("weight_decay", 0.0001, 0.1, log=True),
            }
        else:
            hyperparameter = {
                "n_blocks": 3,
                "d_block": 512,
                "dropout": 0.1,
                "learning_rate": 0.002,
                "weight_decay": 0.0003,
            }

    elif model_type == "mlp":
        if use_hpo:
            hyperparameter = {
                "n_blocks": trial.suggest_int("n_blocks", 1, 5),
                "d_block": trial.suggest_int("d_block", 64, 1024),
                "dropout": trial.suggest_float("dropout", 0.0, 0.5),
                "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.005, log=True),
                "weight_decay": trial.suggest_float("weight_decay", 0.0001, 0.1, log=True),
            }
        else:
            hyperparameter = {
                "n_blocks": 3,
                "d_block": 512,
                "dropout": 0.1,
                "learning_rate": 0.002,
                "weight_decay": 0.0003,
            }

    elif model_type == "random_forest":
        if use_hpo:
            hyperparameter = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 4096),
                "max_depth": trial.suggest_int("max_depth", 5, 100),
                "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
                "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
                "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3, 0.5, 0.7, 1.0]),
                "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
            }
            if task_type == "binary":
                hyperparameter["criterion"] = trial.suggest_categorical("criterion", ["gini", "entropy"])
            else:
                hyperparameter["criterion"] = trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "poisson", "friedman_mse"])
        else:
            hyperparameter = {
                "n_estimators": 1000,
                "max_depth": None,
                "min_samples_split": 2,
                "min_samples_leaf": 1,
            }
            if task_type == "binary":
                hyperparameter["criterion"] = "gini"
                hyperparameter["max_features"] = "sqrt"
            else:
                hyperparameter["criterion"] = "squared_error"
                hyperparameter["max_features"] = 1.0

    return hyperparameter

# Config

In [9]:
config = {
    "data": {
        "datasets": [
                     23381, # Binary classification datasets
                       197, # Regression datasets
                     ],  
        "cache_dir_path": "data/cache/",
        "fold_indices_path": "data/fold_indices/",
        "optuna_db_path": "data/optuna_db/",
        "output_dir_path": "data/output/",
        "outer_folds_path": "data/outer_fold/",
        "results_dir_path": "results/",
    },

    "preprocessing": {
        "threshold_high_cardinality": 10,  
    },

    "model": {
        "teacher_model": "tabpfn",   # Options: 'tabpfn', 'grande', 'catboost', 'tabm', 'mlp', 'random_forest'
        "student_model": "grande", # Options: 'grande', 'catboost', 'tabm', 'mlp', 'random_forest'
    },

    "training": {
        "use_hpo": True,
        "train_on_logits": True,
        "trials": 5,
        "random_state": 42,
        "outer_folds": 5,
        "inner_folds": 2,
    },

    "teacher_models": {
        "tabpfn": {
            "preprocessing": "nn",  
        },
        "catboost": {
            "preprocessing": "tree", 
        },
        "grande": {
            "preprocessing": "grande",  
        },
        "tabm": {
            "preprocessing": "nn",  
        },
        "mlp": {
            "preprocessing": "nn",  
        },
        "random_forest": {
            "preprocessing": "tree",  
        },
    },

    "student_models": {
        "catboost": {
            "preprocessing": "tree",  
        },
        "grande": {
            "preprocessing": "grande",  
        },
        "tabm": {
            "preprocessing": "nn",  
        },
        "mlp": {
            "preprocessing": "nn",  
        },
        "random_forest": {
            "preprocessing": "tree",  
        },
    },
}

# Train Teacher

In [10]:
# Get list of datasets to process from configuration
datasets = config["data"]["datasets"]

# =============================================================================
# MAIN TRAINING LOOP - Process each dataset independently
# =============================================================================
for dataset_id in datasets:
    # -------------------------------------------------------------------------
    # SETUP AND INITIALIZATION
    # -------------------------------------------------------------------------
    logger = setup_logging()
    logger.info("Loading configuration...")

    # Extract model configuration for this run
    model_type = config["model"]["teacher_model"]
    preprocessing_type = config["teacher_models"][model_type]["preprocessing"]
    use_hpo = config["training"]["use_hpo"]

    # -------------------------------------------------------------------------
    # STEP 0: DATA LOADING
    # -------------------------------------------------------------------------
    # Load dataset from OpenML with caching for efficiency
    X, y, cat_cols, _, task_type = load_dataset(
        dataset_id=dataset_id,
        config=config,
    )

    # -------------------------------------------------------------------------
    # STEP 1: INFRASTRUCTURE SETUP
    # -------------------------------------------------------------------------
    # Note: Checking for existing results is placeholder for future implementation
    summary_file = os.path.join(config["data"]["results_dir_path"], "teacher", f"{dataset_id}_results.json")
    if os.path.exists(summary_file):
        with open(summary_file, 'r') as f:
            existing_results = json.load(f)
        
        # Check if we already have results with the same configuration
        config_exists = False
        for key, result in existing_results.items():
            if (result.get('model_type') == model_type and 
                result.get('use_hpo') == use_hpo and 
                result.get('seed') == config["training"]["random_state"]):
                config_exists = True
                logger.info(f"Results already exist for dataset {dataset_id} with model {model_type}, HPO: {use_hpo}, seed: {config['training']['random_state']}")
                break
        
        if config_exists:
            logger.info(f"Skipping dataset {dataset_id} - results already computed")
            continue

    # Configure GPU/CPU usage for training
    device = check_GPU_availability()

    # Set random seed for reproducibility across all libraries
    set_seed(config["training"]["random_state"])
    logger.info(f"Random seed set to {config['training']['random_state']}")

    # -------------------------------------------------------------------------
    # STEP 2: INITIALIZE DATA STRUCTURES
    # -------------------------------------------------------------------------
    # List to store predictions from each outer fold
    output_dfs = []

    outer_fold_scores = []
    
    # Dictionary to store all fold indices for reproducibility and student training
    # Structure: {"outer_folds": {fold_id: {train_idx, test_idx}},
    #            "inner_folds": {outer_fold_id: {inner_fold_id: {train_idx, val_idx}}}}
    fold_indices = {
        "outer_folds": {},
        "inner_folds": {}
    }

    # -------------------------------------------------------------------------
    # STEP 3: OUTER CROSS-VALIDATION SETUP
    # -------------------------------------------------------------------------
    # Choose appropriate CV strategy based on task type to maintain class balance
    if task_type == "binary":
        outer_cv = StratifiedKFold(
            n_splits=config["training"]["outer_folds"],
            shuffle=True,
            random_state=config["training"]["random_state"],
        )
    else:
        outer_cv = KFold(
            n_splits=config["training"]["outer_folds"],
            shuffle=True,
            random_state=config["training"]["random_state"],
        )

    # =========================================================================
    # STEP 4: OUTER CROSS-VALIDATION LOOP
    # =========================================================================
    # Each iteration provides one unbiased performance estimate
    for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(X, y), start=1):

        # Log the first 10 indices of the outer fold for debugging
        logger.info(f"Outer Fold {outer_fold} - Train Indices: {train_idx[:10]}, Test Indices: {test_idx[:10]}")

        logger.info(f"-------------------- Outer Fold {outer_fold} --------------------")
        
        # ---------------------------------------------------------------------
        # FOLD INDEX MANAGEMENT
        # ---------------------------------------------------------------------
        # Store outer fold indices for later use in student training and validation
        fold_indices["outer_folds"][f"fold_{outer_fold}"] = {
            "train_idx": train_idx.tolist(),
            "test_idx": test_idx.tolist()
        }

        # Split data according to current outer fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # ---------------------------------------------------------------------
        # INNER CROSS-VALIDATION SETUP (for model validation)
        # ---------------------------------------------------------------------
        # Choose appropriate CV strategy for inner folds
        if task_type == "binary":
            inner_cv = StratifiedKFold(
                n_splits=config["training"]["inner_folds"],
                shuffle=True,
                random_state=config["training"]["random_state"],
            )
        else:
            inner_cv = KFold(
                n_splits=config["training"]["inner_folds"],
                shuffle=True,
                random_state=config["training"]["random_state"],
            )
        
        # Initialize storage for inner fold indices within this outer fold
        fold_indices["inner_folds"][f"outer_fold_{outer_fold}"] = {}

        # =====================================================================
        # STEP 5: INNER CROSS-VALIDATION LOOP (hyperparameter validation) 
        # =====================================================================
        def objective(trial):

            hyperparams = suggest_hyperparameters(
                trial=trial,
                model_type=model_type,
                task_type=task_type,
                use_hpo=use_hpo,
            )

            inner_fold_scores = []
            val_metrics_list = []

            # This loop would typically be used for hyperparameter optimization
            for inner_fold, (inner_train_index, inner_val_index) in enumerate(inner_cv.split(X_train, y_train), start=1):
                
                # -----------------------------------------------------------------
                # INDEX MANAGEMENT (Critical for avoiding data leakage)
                # -----------------------------------------------------------------
                # Convert relative indices (within outer training set) to absolute indices
                absolute_inner_train_idx = train_idx[inner_train_index]
                absolute_inner_val_idx = train_idx[inner_val_index]
                
                # Store inner fold indices using absolute indices for consistency
                # Only store indices for the first trial
                if trial.number == 0:  
                    fold_indices["inner_folds"][f"outer_fold_{outer_fold}"][f"inner_fold_{inner_fold}"] = {
                        "train_idx": absolute_inner_train_idx.tolist(),
                        "val_idx": absolute_inner_val_idx.tolist()
                    }

                # Split inner training data using relative indices
                X_inner_train, X_inner_val = X_train.iloc[inner_train_index], X_train.iloc[inner_val_index]
                y_inner_train, y_inner_val = y_train[inner_train_index], y_train[inner_val_index]

                # -----------------------------------------------------------------
                # PREPROCESSING: Apply model-specific data transformations
                # -----------------------------------------------------------------
                X_inner_train, X_inner_val = preprocess(
                    X_inner_train,
                    y_inner_train,
                    X_inner_val, 
                    cat_cols,
                    config,
                    preprocessing_type=preprocessing_type,
                )

                # -----------------------------------------------------------------
                # MODEL TRAINING: Train teacher model on inner training data
                # -----------------------------------------------------------------
                model = get_teacher_model(config=config, task_type=task_type, device=device, hyperparams=hyperparams, cat_cols=cat_cols)
                logger.info(f"Training Model on Outer Fold {outer_fold}, Inner Fold {inner_fold}...")
                model.train(X_inner_train, y_inner_train)

                # -----------------------------------------------------------------
                # VALIDATION: Evaluate model performance on inner validation set
                # -----------------------------------------------------------------
                val_preds = model.predict(X_inner_val)
                val_metrics = model.evaluate(val_preds, y_inner_val)

                # Store metrics from this fold for later mean calculation
                val_metrics_list.append(val_metrics)

                if task_type == "binary":
                    inner_fold_scores.append(val_metrics["f1"])
                    trial.report(np.mean(inner_fold_scores), step=inner_fold)
                else:
                    inner_fold_scores.append(-val_metrics["mae"])
                    trial.report(np.mean(inner_fold_scores), step=inner_fold)

                # Check if the trial should be pruned (With 2 inner folds, pruning not recommended)
                # if trial.should_prune():
                #     logger.info("Trial pruned.")
                #     raise optuna.TrialPruned()

            # Calculate and set mean metrics as user attributes after all inner folds
            if val_metrics_list:
                # Get all metric keys from the first fold
                metric_keys = val_metrics_list[0].keys()
                
                for metric_key in metric_keys:
                    # Calculate mean across all inner folds for this metric
                    metric_values = [fold_metrics[metric_key] for fold_metrics in val_metrics_list]
                    mean_metric = np.mean(metric_values)
                    trial.set_user_attr(f"mean_{metric_key}", mean_metric)

            return np.mean(inner_fold_scores)
            
        logger.info(f"Starting hyperparameter optimization for outer fold {outer_fold}...")

        study_kwargs = dict(
            direction="maximize",
            study_name=f"{dataset_id}.{outer_fold}.{model_type}",
            load_if_exists=True,
            sampler=optuna.samplers.TPESampler(seed=config["training"]["random_state"]),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=1),
        )

        if use_hpo:
            os.makedirs(config["data"]["optuna_db_path"], exist_ok=True)
            study_kwargs["storage"] = f"sqlite:///{config['data']['optuna_db_path']}/optuna.db"

        study = optuna.create_study(
            **study_kwargs,
        )

        # Optimize
        completed_trials = len(study.trials)
        remaining_trials = config["training"]["trials"] - completed_trials

        if remaining_trials > 0:
            default_hyperparams = suggest_hyperparameters(None, model_type, task_type, False)
            study.enqueue_trial(default_hyperparams)
            study.optimize(
                objective, 
                n_trials=remaining_trials if use_hpo else 1,
                show_progress_bar=False
            )
        else:
            logger.info("The study has already reached the maximum number of trials.")

        if use_hpo:
            # When HPO is used, trial.suggest_ methods are called, populating trial.params
            best_hyperparams = study.best_trial.params 
        else:
            # When not using HPO, a single trial runs with default parameters.
            best_hyperparams = suggest_hyperparameters(trial=None, model_type=model_type, task_type=task_type, use_hpo=False)

        logger.info(f"Best hyperparameters for fold {outer_fold}: {best_hyperparams}")
        logger.info(f"Best score: {study.best_value:.4f}")

        # =====================================================================
        # STEP 6: FINAL MODEL TRAINING (on complete outer training set)
        # =====================================================================        
        # Apply same preprocessing to outer training and test sets
        X_train, X_test = preprocess(
            X_train,
            y_train,
            X_test,
            cat_cols,
            config,
            preprocessing_type=preprocessing_type,  
        )
        
        # Train final model on complete outer training set
        logger.info("------------------------------------------------------")
        logger.info(f"Retraining Model on Outer Fold {outer_fold}")

        model = get_teacher_model(
            config=config,
            task_type=task_type,
            device=device,
            hyperparams=best_hyperparams,  # Use best hyperparameters from HPO
            cat_cols=cat_cols,  
        )
        model.train(X_train, y_train)

        # =====================================================================
        # STEP 7: FINAL EVALUATION (unbiased performance on outer test set)
        # =====================================================================
        # This provides the unbiased performance estimate for this fold
        start_time = time.time()
        test_preds = model.predict(X_test)
        end_time = time.time() - start_time
        logger.info(f"\t Inference Time: {end_time:.5f} seconds")
        test_metrics = model.evaluate(test_preds, y_test)

        # Store outer fold score for later analysis
        outer_fold_results = {
            "fold": outer_fold,
            "seed": config["training"]["random_state"],
            "inference_time": end_time,
            **test_metrics,
        }
        outer_fold_scores.append(outer_fold_results)

        # =====================================================================
        # STEP 8: STORE PREDICTIONS FOR STUDENT TRAINING
        # =====================================================================
        # Save predictions with their corresponding dataset indices
        # These will be used as targets for training student models
        output_dfs.append(pd.DataFrame({
            "index": test_idx,
            "output": test_preds[:, 1] if task_type == "binary" else test_preds
        }))
    
    # =========================================================================
    # STEP 9: SAVE RESULTS AND METADATA
    # =========================================================================

    # -------------------------------------------------------------------------
    # SAVE OUTER FOLD METRICS
    # -------------------------------------------------------------------------
    outer_fold_df = pd.DataFrame(outer_fold_scores)
    sub_folder = "hpo" if use_hpo else "default"
    output_dir = os.path.join(config["data"]["outer_folds_path"], "teacher", sub_folder)
    os.makedirs(output_dir, exist_ok=True)

    metrics_file = os.path.join(output_dir, f"{dataset_id}_{model_type}.csv")
    outer_fold_df.to_csv(metrics_file, index=False)
    logger.info(f"Outer fold metrics saved to: {metrics_file}")

    # Calculate and log overall performance across all folds
    mean_inference_time = outer_fold_df['inference_time'].mean()
    std_inference_time = outer_fold_df['inference_time'].std()
    mean_parameters = outer_fold_df['parameters'].mean()
    std_parameters = outer_fold_df['parameters'].std()

    if task_type == "binary":
        mean_acc = outer_fold_df['acc'].mean()
        std_acc = outer_fold_df['acc'].std()
        mean_f1 = outer_fold_df['f1'].mean()
        std_f1 = outer_fold_df['f1'].std()
        mean_roc = outer_fold_df['roc'].mean()
        std_roc = outer_fold_df['roc'].std()
        
        logger.info(f"=== FINAL RESULTS FOR DATASET {dataset_id} ===")
        logger.info(f"Balanced Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
        logger.info(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")
        logger.info(f"ROC AUC: {mean_roc:.4f} ± {std_roc:.4f}")
    else:
        mean_mae = outer_fold_df['mae'].mean()
        std_mae = outer_fold_df['mae'].std()
        mean_mse = outer_fold_df['mse'].mean()
        std_mse = outer_fold_df['mse'].std()
        mean_rmse = outer_fold_df['rmse'].mean()
        std_rmse = outer_fold_df['rmse'].std()
        mean_r2 = outer_fold_df['r2'].mean()
        std_r2 = outer_fold_df['r2'].std()

        
        logger.info(f"=== FINAL RESULTS FOR DATASET {dataset_id} ===")
        logger.info(f"MAE: {mean_mae:.4f} ± {std_mae:.4f}")
        logger.info(f"MSE: {mean_mse:.4f} ± {std_mse:.4f}")
        logger.info(f"RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
        logger.info(f"R2: {mean_r2:.4f} ± {std_r2:.4f}")
    
    logger.info(f"Mean Inference Time: {mean_inference_time:.4f} ± {std_inference_time:.4f}")
    logger.info(f"Mean Parameters: {mean_parameters:.4f} ± {std_parameters:.4f}")

    # Save summary statistics as well
    summary_stats = {
        "dataset_id": dataset_id,
        "model_type": model_type,
        "task_type": task_type,
        "seed": config["training"]["random_state"],
        "use_hpo": config["training"]["use_hpo"],
        "mean_inference_time": mean_inference_time,
        "std_inference_time": std_inference_time,
        "mean_parameters": mean_parameters,
        "std_parameters": std_parameters,
    }

    if task_type == "binary":
        summary_stats.update({
            "mean_acc": mean_acc,
            "std_acc": std_acc,
            "mean_f1": mean_f1,
            "std_f1": std_f1,
            "mean_roc": mean_roc,
            "std_roc": std_roc,
        })
    else:
        summary_stats.update({
            "mean_mae": mean_mae,
            "std_mae": std_mae,
            "mean_mse": mean_mse,
            "std_mse": std_mse,
            "mean_rmse": mean_rmse,
            "std_rmse": std_rmse,
            "mean_r2": mean_r2,
            "std_r2": std_r2,
        })

    # Load existing summary file if it exists, otherwise create new
    summary_file = os.path.join(config["data"]["results_dir_path"], "teacher", f"{dataset_id}_results.json")
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(summary_file), exist_ok=True)
    if os.path.exists(summary_file):
        with open(summary_file, 'r') as f:
            all_results = json.load(f)
    else:
        all_results = {}

    # Add current model results to the dataset summary
    # Use simple incremental numbering
    next_num = len(all_results) + 1
    model_key = str(next_num)

    # Add current model results to the dataset summary
    all_results[model_key] = summary_stats

    # Save updated summary
    with open(summary_file, 'w') as f:
        json.dump(all_results, f, indent=2)
    logger.info(f"Summary statistics saved to: {summary_file}")

    # -------------------------------------------------------------------------
    # SAVE FOLD INDICES (for reproducibility and student training)
    # -------------------------------------------------------------------------
    fold_indices_file = os.path.join(config["data"]["fold_indices_path"], f"dataset_{dataset_id}.json")
    # Create the directory if it doesn't exist
    os.makedirs(config["data"]["fold_indices_path"], exist_ok=True)
    if not os.path.exists(fold_indices_file):
        with open(fold_indices_file, 'w') as f:
            json.dump(fold_indices, f, indent=2)
        logger.info(f"Fold indices saved to: {fold_indices_file}")
    else:
        logger.info(f"Fold indices file already exists: {fold_indices_file}")

    # -------------------------------------------------------------------------
    # SAVE TEACHER PREDICTIONS (targets for student training)
    # -------------------------------------------------------------------------
    # Combine predictions from all outer folds
    if output_dfs:  # Check if we have any DataFrames to concatenate
        output_df = pd.concat(output_dfs, ignore_index=True)
    else:
        output_df = pd.DataFrame(columns=["index", "output"])
    output_df = output_df.sort_values(by="index")
    
    # Create the full directory structure
    sub_folder = "hpo" if use_hpo else "default"
    output_dir = os.path.join(config["data"]["output_dir_path"], sub_folder, "teacher")
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, f"{dataset_id}_{model_type}.csv")
    output_df.to_csv(output_file, index=False)
    logger.info(f"{config['model']['teacher_model']} outputs saved to: {output_file}")

2025-06-06 11:33:31,467 - __main__ - INFO - Loading configuration...
2025-06-06 11:33:31,468 - __main__ - INFO - Loading dataset 23381 from cache at 'data/cache/openml_dataset_23381.pkl'...
2025-06-06 11:33:31,470 - __main__ - INFO - Dataset 23381 loaded successfully with task type: binary


2025-06-06 11:33:31,622 - __main__ - INFO - Using GPU: NVIDIA GeForce RTX 2080 Ti
2025-06-06 11:33:31,635 - __main__ - INFO - Random seed set to 42
2025-06-06 11:33:31,637 - __main__ - INFO - Outer Fold 1 - Train Indices: [ 1  3  5  6  7  8  9 10 11 12], Test Indices: [ 0  2  4 18 19 27 35 39 57 59]
2025-06-06 11:33:31,637 - __main__ - INFO - -------------------- Outer Fold 1 --------------------
2025-06-06 11:33:31,639 - __main__ - INFO - Starting hyperparameter optimization for outer fold 1...
[I 2025-06-06 11:33:32,544] A new study created in RDB with name: 23381.1.tabpfn
2025-06-06 11:33:32,603 - __main__ - INFO - Preprocessing pipeline for Neural Network model.
/home/mherre/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 2, 3, 4, 5, 6, 7, 8, 9, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
2025-06-06 11:33:34,475 - __main__ - INFO - Trainin

# Train Student

In [ ]:
# Get list of datasets to process from configuration
datasets = config["data"]["datasets"]

# =============================================================================
# MAIN TRAINING LOOP - Process each dataset independently
# =============================================================================
for dataset_id in datasets:
    # -------------------------------------------------------------------------
    # SETUP AND INITIALIZATION
    # -------------------------------------------------------------------------
    logger = setup_logging()
    logger.info("Loading configuration...")

    # Extract model configuration for this run
    student_model_type = config["model"]["student_model"]
    teacher_model_type = config["model"]["teacher_model"]
    preprocessing_type = config["student_models"][student_model_type]["preprocessing"]
    use_hpo = config["training"]["use_hpo"]
    train_on_logits = config["training"]["train_on_logits"]

    # -------------------------------------------------------------------------
    # STEP 0: DATA LOADING
    # -------------------------------------------------------------------------
    # Load dataset from OpenML with caching for efficiency
    X, y, cat_cols, _, original_task_type = load_dataset(
        dataset_id=dataset_id,
        config=config,
    )

    # -------------------------------------------------------------------------
    # STEP 1: INFRASTRUCTURE SETUP
    # -------------------------------------------------------------------------
    # Note: Checking for existing results is placeholder for future implementation
    model_task_type = "binary" if (original_task_type == "binary" and not train_on_logits) else "regression"
    summary_file = os.path.join(config["data"]["results_dir_path"], "student", f"{dataset_id}_results.json")
    if os.path.exists(summary_file):
        with open(summary_file, 'r') as f:
            existing_results = json.load(f)
        
        # Check if we already have results with the same configuration
        config_exists = False
        for key, result in existing_results.items():
            if (result.get('student_model_type') == student_model_type and 
                result.get('teacher_model_type') == teacher_model_type and
                result.get('student_task_type') == model_task_type and
                result.get('use_hpo') == use_hpo and 
                result.get('seed') == config["training"]["random_state"]):
                config_exists = True
                logger.info(f"Results already exist for dataset {dataset_id} with model {student_model_type}({teacher_model_type}), HPO: {use_hpo}, seed: {config['training']['random_state']}")
                break
        
        if config_exists:
            logger.info(f"Skipping dataset {dataset_id} - results already computed")
            continue

    # Configure GPU/CPU usage for training
    device = check_GPU_availability()

    # Set random seed for reproducibility across all libraries
    set_seed(config["training"]["random_state"])
    logger.info(f"Random seed set to {config['training']['random_state']}")

    # -------------------------------------------------------------------------
    # STEP 2: LOAD PREVIOUSLY SAVED FOLD INDICES AND TEACHER PREDICTIONS
    # -------------------------------------------------------------------------
    # Load the saved fold indices 
    fold_indices_file = os.path.join(config["data"]["fold_indices_path"], f"dataset_{dataset_id}.json")
    with open(fold_indices_file, 'r') as f:
        fold_indices = json.load(f)
    logger.info(f"Loaded fold indices from: {fold_indices_file}") 

    # Load the TabPFN outputs (teacher predictions)
    subfolder = "hpo" if config["training"]["use_hpo"] else "default"
    directory = os.path.join(config["data"]["output_dir_path"], subfolder, "teacher")
    teacher_outputs_file = os.path.join(directory, f"{dataset_id}_{teacher_model_type}.csv")
    teacher_outputs_df = pd.read_csv(teacher_outputs_file)
    logger.info(f"Loaded {teacher_model_type} outputs from: {teacher_outputs_file}")

    # --------------------------------------------------------------------------
    # STEP 3: PREPROCESS TEACHER OUTPUTS
    # --------------------------------------------------------------------------
    if original_task_type == "regression":
        model_task_type = "regression"
        # For regression, we can use the outputs directly as they are already numeric
        teacher_preds = teacher_outputs_df['output'].astype(float)
        # Create a mapping from index to outputs for easy lookup
        index_to_outputs = dict(zip(teacher_outputs_df['index'].values, teacher_preds))

    else:
        if train_on_logits:
            model_task_type = "regression"
            # Convert probabilities to logits
            # Clip probabilities to avoid log(0) or log(1)
            eps = 1e-7
            teacher_probs = np.clip(teacher_outputs_df['output'].values, eps, 1 - eps)
            teacher_logits = np.log(teacher_probs / (1 - teacher_probs))
            # Create a mapping from index to logits for easy lookup
            index_to_outputs = dict(zip(teacher_outputs_df['index'].values, teacher_logits))
        else:
            model_task_type = "binary"
            # Convert probabilities to binary predictions
            teacher_preds = (teacher_outputs_df['output'].values > 0.5).astype(int)
            # Create a mapping from index to predictions for easy lookup
            index_to_outputs = dict(zip(teacher_outputs_df['index'].values, teacher_preds))

    # -------------------------------------------------------------------------
    # STEP 4: INITIALIZE DATA STRUCTURES
    # -------------------------------------------------------------------------
    output_dfs = []

    outer_fold_scores = []

    # =========================================================================
    # STEP 5: OUTER CROSS-VALIDATION LOOP
    # =========================================================================
    # Each iteration provides one unbiased performance estimate
    for fold_key, fold_data in fold_indices["outer_folds"].items():

        # Extract outer fold number and indices from the fold key
        outer_fold = int(fold_key.split('_')[1])
        train_idx = np.array(fold_data["train_idx"])
        test_idx = np.array(fold_data["test_idx"])

        # Log the first 10 indices of the outer fold for debugging
        logger.info(f"Outer Fold {outer_fold} - Train Indices: {train_idx[:10]}, Test Indices: {test_idx[:10]}")

        logger.info(f"-------------------- Outer Fold {outer_fold} --------------------")

        # Split data according to current outer fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # ---------------------------------------------------------------------
        # INNER CROSS-VALIDATION SETUP (for model validation)
        # ---------------------------------------------------------------------
        inner_folds_data = fold_indices["inner_folds"][f"outer_fold_{outer_fold}"]

        # =========================================================================
        # STEP 6: INNER CROSS-VALIDATION LOOP (hyperparameter validation)
        # =========================================================================
        def objective(trial):
            
            hyperparams = suggest_hyperparameters(
                trial=trial,
                model_type=student_model_type,
                task_type=model_task_type,
                use_hpo=use_hpo,
            )

            inner_fold_scores = []
            val_metrics_list = []

            # This loop would typically be used for hyperparameter optimization
            for inner_fold_key, inner_fold_data in inner_folds_data.items():
                inner_fold = int(inner_fold_key.split('_')[2])
                
                # -----------------------------------------------------------------
                # INDEX MANAGEMENT (Critical for avoiding data leakage)
                # -----------------------------------------------------------------
                # Get absolute indices from saved data
                absolute_inner_train_idx = np.array(inner_fold_data["train_idx"])
                absolute_inner_val_idx = np.array(inner_fold_data["val_idx"])
                
                # Convert absolute indices to relative indices for the current outer training set
                inner_train_relative = np.where(np.isin(train_idx, absolute_inner_train_idx))[0]
                inner_val_relative = np.where(np.isin(train_idx, absolute_inner_val_idx))[0] 

                # Split inner training data using relative indices
                X_inner_train, X_inner_val = X_train.iloc[inner_train_relative], X_train.iloc[inner_val_relative]
                y_inner_train, y_inner_val = y_train[inner_train_relative], y_train[inner_val_relative] # Hard labels or Regression targets

                # Get teacher outputs for inner training and validation sets
                teacher_outputs_inner_train = np.array([index_to_outputs[idx] for idx in absolute_inner_train_idx])
                teacher_outputs_inner_val = np.array([index_to_outputs[idx] for idx in absolute_inner_val_idx]) # Probs or Regression logits

                # ---------------------------------------------------------------------
                # PREPROCESSING: Apply model-specific data transformations
                # ---------------------------------------------------------------------
                X_inner_train, X_inner_val = preprocess(
                    X_inner_train,
                    y_inner_train,
                    X_inner_val, 
                    cat_cols,
                    config,
                    preprocessing_type=preprocessing_type,
                )
                # ----------------------------------------------------------------------
                # MODEL TRAINING: Train student model on inner training data
                # ----------------------------------------------------------------------
                model = get_student_model(config=config, task_type=model_task_type, device=device, hyperparams=hyperparams, cat_cols=cat_cols)
                logger.info(f"Training Model on Outer Fold {outer_fold}, Inner Fold {inner_fold}...")
                model.train(X_inner_train, teacher_outputs_inner_train)

                # ----------------------------------------------------------------------
                # VALIDATION: Evaluate model performance on inner validation set
                # ----------------------------------------------------------------------
                val_preds = model.predict(X_inner_val) # Probs or Regression logits
                val_metrics = model.evaluate(y_pred=val_preds, y_true=y_inner_val, y_teacher_true=teacher_outputs_inner_val, original_task_type=original_task_type)

                # Store metrics from this fold for later mean calculation
                val_metrics_list.append(val_metrics)

                if model_task_type == "binary":
                    inner_fold_scores.append(val_metrics["fidelity_f1"])
                    trial.report(np.mean(inner_fold_scores), step=inner_fold)
                else:
                    inner_fold_scores.append(-val_metrics["fidelity_mae"])
                    trial.report(np.mean(inner_fold_scores), step=inner_fold)

            # Calculate mean metrics as user attributes after all inner folds
            if val_metrics_list:
                # Get all metric keys from the first fold
                metric_keys = val_metrics_list[0].keys()
                
                for metric_key in metric_keys:
                    # Calculate mean across all inner folds for this metric
                    metric_values = [fold_metrics[metric_key] for fold_metrics in val_metrics_list]
                    mean_metric = np.mean(metric_values)
                    trial.set_user_attr(f"mean_{metric_key}", mean_metric)

            return np.mean(inner_fold_scores)
        
        logger.info(f"Starting hyperparameter optimization for outer fold {outer_fold}...")

        study_kwargs = dict(
            direction="maximize",
            study_name=f"{dataset_id}.{outer_fold}.{student_model_type}({teacher_model_type}).{model_task_type[:2]}.{original_task_type[:2]}",
            load_if_exists=True,
            sampler=optuna.samplers.TPESampler(seed=config["training"]["random_state"]),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=1),
        )

        if use_hpo:
            os.makedirs(config["data"]["optuna_db_path"], exist_ok=True)
            study_kwargs["storage"] = f"sqlite:///{config['data']['optuna_db_path']}/optuna.db"

        study = optuna.create_study(
            **study_kwargs,
        )

        # Optimize
        completed_trials = len(study.trials)
        remaining_trials = config["training"]["trials"] - completed_trials


        if remaining_trials > 0:
            default_hyperparams = suggest_hyperparameters(None, student_model_type, model_task_type, False)
            study.enqueue_trial(default_hyperparams)
            study.optimize(
                objective, 
                n_trials=remaining_trials if use_hpo else 1,
                show_progress_bar=False
            )
        else:
            logger.info("The study has already reached the maximum number of trials.")

        if use_hpo:
            # When HPO is used, trial.suggest_ methods are called, populating trial.params
            best_hyperparams = study.best_trial.params
        else:
            # When not using HPO, a single trial runs with default parameters.
            best_hyperparams = suggest_hyperparameters(trial=None, model_type=student_model_type, task_type=model_task_type, use_hpo=False)

        logger.info(f"Best hyperparameters for fold {outer_fold}: {best_hyperparams}")
        logger.info(f"Best score: {study.best_value:.4f}")

        # =========================================================================
        # STEP 7: FINAL MODEL TRAINING (on complete outer training set)
        # =========================================================================
        # Get teacher logits for outer training set
        teacher_outputs_train = np.array([index_to_outputs[idx] for idx in train_idx])
        # Preprocess the outer training and test sets
        X_train, X_test = preprocess(
            X_train,
            y_train,
            X_test,
            cat_cols,
            config,
            preprocessing_type=preprocessing_type,  
        )
        
        # Train final model on complete outer training set
        logger.info("------------------------------------------------------")
        logger.info(f"Retraining Model on Outer Fold {outer_fold}")
        model = get_student_model(
            config=config,
            task_type=model_task_type,
            device=device,
            hyperparams=best_hyperparams,  # Use best hyperparameters from HPO
            cat_cols=cat_cols,  
        )
        model.train(X_train, teacher_outputs_train)

        # =========================================================================
        # STEP 8: FINAL EVALUATION (unbiased performance on outer test set)
        # =========================================================================
        # This provides the unbiased performance estimate for this fold
        start_time = time.time()
        test_preds = model.predict(X_test)  # Predicted logits or probabilities
        end_time = time.time() - start_time
        logger.info(f"\t Inference Time: {end_time:.5f} seconds")
        teacher_outputs_test = np.array([index_to_outputs[idx] for idx in test_idx])
        test_metrics = model.evaluate(test_preds, y_test, teacher_outputs_test, original_task_type)

        # Store outer fold score for later analysis
        outer_fold_results = {
            "fold": outer_fold,
            "seed": config["training"]["random_state"],
            "inference_time": end_time,
            **test_metrics,
        }
        outer_fold_scores.append(outer_fold_results)

        # =========================================================================
        # STEP 9: STORE PREDICTIONS FOR STUDENT TRAINING
        # =========================================================================
        # Save predictions with their corresponding dataset indices
        output_dfs.append(pd.DataFrame({
            "index": test_idx,
            "output": test_preds[:, 1] if model_task_type == "binary" else test_preds  
        }))
    
    # =========================================================================
    # STEP 10: SAVE RESULTS AND METADATA
    # =========================================================================

    # -------------------------------------------------------------------------
    # SAVE OUTER FOLD METRICS
    # -------------------------------------------------------------------------
    outer_fold_df = pd.DataFrame(outer_fold_scores)
    sub_folder = "hpo" if use_hpo else "default"
    output_dir = os.path.join(config["data"]["outer_folds_path"], "student", sub_folder)
    os.makedirs(output_dir, exist_ok=True)

    metrics_file = os.path.join(output_dir, f"{dataset_id}_{student_model_type}({teacher_model_type})_{model_task_type[:2]}.csv")
    outer_fold_df.to_csv(metrics_file, index=False)
    logger.info(f"Outer fold metrics saved to: {metrics_file}")

    # Calculate and log overall performance across all folds
    mean_inference_time = outer_fold_df['inference_time'].mean()
    std_inference_time = outer_fold_df['inference_time'].std()
    mean_parameters = outer_fold_df['parameters'].mean()
    std_parameters = outer_fold_df['parameters'].std()

    if original_task_type == "binary":
        mean_acc = outer_fold_df['acc'].mean()
        std_acc = outer_fold_df['acc'].std()
        mean_f1 = outer_fold_df['f1'].mean()
        std_f1 = outer_fold_df['f1'].std()
        mean_roc = outer_fold_df['roc'].mean()
        std_roc = outer_fold_df['roc'].std()
        
        mean_fidelity_acc = outer_fold_df['fidelity_acc'].mean()
        std_fidelity_acc = outer_fold_df['fidelity_acc'].std()
        mean_fidelity_f1 = outer_fold_df['fidelity_f1'].mean()
        std_fidelity_f1 = outer_fold_df['fidelity_f1'].std()
        mean_fidelity_roc = outer_fold_df['fidelity_roc'].mean()
        std_fidelity_roc = outer_fold_df['fidelity_roc'].std()
        mean_fidelity_kl_div = outer_fold_df['fidelity_kl_div'].mean()
        std_fidelity_kl_div = outer_fold_df['fidelity_kl_div'].std()
        
        logger.info(f"=== FINAL RESULTS FOR DATASET {dataset_id} ===")
        logger.info(f"Balanced Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
        logger.info(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")
        logger.info(f"ROC AUC: {mean_roc:.4f} ± {std_roc:.4f}")
        logger.info(f"Fidelity Accuracy: {mean_fidelity_acc:.4f} ± {std_fidelity_acc:.4f}")
        logger.info(f"Fidelity F1 Score: {mean_fidelity_f1:.4f} ± {std_fidelity_f1:.4f}")
        logger.info(f"Fidelity ROC AUC: {mean_fidelity_roc:.4f} ± {std_fidelity_roc:.4f}")
        logger.info(f"Fidelity KL Divergence: {mean_fidelity_kl_div:.4f} ± {std_fidelity_kl_div:.4f}")

        if model_task_type == "regression":
            mean_fidelity_mae = outer_fold_df['fidelity_mae'].mean()
            std_fidelity_mae = outer_fold_df['fidelity_mae'].std()
            mean_fidelity_mse = outer_fold_df['fidelity_mse'].mean()
            std_fidelity_mse = outer_fold_df['fidelity_mse'].std()
            mean_fidelity_rmse = outer_fold_df['fidelity_rmse'].mean()
            std_fidelity_rmse = outer_fold_df['fidelity_rmse'].std()
            mean_fidelity_r2 = outer_fold_df['fidelity_r2'].mean()
            std_fidelity_r2 = outer_fold_df['fidelity_r2'].std()

            logger.info(f"Fidelity MAE: {mean_fidelity_mae:.4f} ± {std_fidelity_mae:.4f}")
            logger.info(f"Fidelity MSE: {mean_fidelity_mse:.4f} ± {std_fidelity_mse:.4f}")
            logger.info(f"Fidelity RMSE: {mean_fidelity_rmse:.4f} ± {std_fidelity_rmse:.4f}")
            logger.info(f"Fidelity R2: {mean_fidelity_r2:.4f} ± {std_fidelity_r2:.4f}")

    else:
        mean_mae = outer_fold_df['mae'].mean()
        std_mae = outer_fold_df['mae'].std()
        mean_mse = outer_fold_df['mse'].mean()
        std_mse = outer_fold_df['mse'].std()
        mean_rmse = outer_fold_df['rmse'].mean()
        std_rmse = outer_fold_df['rmse'].std()
        mean_r2 = outer_fold_df['r2'].mean()
        std_r2 = outer_fold_df['r2'].std()

        logger.info(f"=== FINAL RESULTS FOR DATASET {dataset_id} ===")
        logger.info(f"MAE: {mean_mae:.4f} ± {std_mae:.4f}")
        logger.info(f"MSE: {mean_mse:.4f} ± {std_mse:.4f}")
        logger.info(f"RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
        logger.info(f"R2: {mean_r2:.4f} ± {std_r2:.4f}")

        mean_fidelity_mae = outer_fold_df['fidelity_mae'].mean()
        std_fidelity_mae = outer_fold_df['fidelity_mae'].std()
        mean_fidelity_mse = outer_fold_df['fidelity_mse'].mean()
        std_fidelity_mse = outer_fold_df['fidelity_mse'].std()
        mean_fidelity_rmse = outer_fold_df['fidelity_rmse'].mean()
        std_fidelity_rmse = outer_fold_df['fidelity_rmse'].std()
        mean_fidelity_r2 = outer_fold_df['fidelity_r2'].mean()
        std_fidelity_r2 = outer_fold_df['fidelity_r2'].std()

        logger.info(f"Fidelity MAE: {mean_fidelity_mae:.4f} ± {std_fidelity_mae:.4f}")
        logger.info(f"Fidelity MSE: {mean_fidelity_mse:.4f} ± {std_fidelity_mse:.4f}")
        logger.info(f"Fidelity RMSE: {mean_fidelity_rmse:.4f} ± {std_fidelity_rmse:.4f}")
        logger.info(f"Fidelity R2: {mean_fidelity_r2:.4f} ± {std_fidelity_r2:.4f}")

    logger.info(f"Mean Inference Time: {mean_inference_time:.4f} ± {std_inference_time:.4f}")
    logger.info(f"Mean Parameters: {mean_parameters:.4f} ± {std_parameters:.4f}")

    # Save summary statistics as well
    summary_stats = {
        "dataset_id": dataset_id,
        "student_model_type": student_model_type,
        "teacher_model_type": teacher_model_type,
        "original_task_type": original_task_type,
        "student_task_type": model_task_type,
        "seed": config["training"]["random_state"],
        "use_hpo": config["training"]["use_hpo"],
        "mean_inference_time": mean_inference_time,
        "std_inference_time": std_inference_time,
        "mean_parameters": mean_parameters,
        "std_parameters": std_parameters,
    }

    if original_task_type == "binary":
        summary_stats.update({
            "mean_acc": mean_acc,
            "std_acc": std_acc,
            "mean_f1": mean_f1,
            "std_f1": std_f1,
            "mean_roc": mean_roc,
            "std_roc": std_roc,
            "mean_fidelity_acc": mean_fidelity_acc,
            "std_fidelity_acc": std_fidelity_acc,
            "mean_fidelity_f1": mean_fidelity_f1,
            "std_fidelity_f1": std_fidelity_f1,
            "mean_fidelity_roc": mean_fidelity_roc,
            "std_fidelity_roc": std_fidelity_roc,
            "mean_fidelity_kl_div": mean_fidelity_kl_div,
            "std_fidelity_kl_div": std_fidelity_kl_div,
        })
        if model_task_type == "regression":
            summary_stats.update({
                "mean_fidelity_mae": mean_fidelity_mae,
                "std_fidelity_mae": std_fidelity_mae,
                "mean_fidelity_mse": mean_fidelity_mse,
                "std_fidelity_mse": std_fidelity_mse,
                "mean_fidelity_rmse": mean_fidelity_rmse,
                "std_fidelity_rmse": std_fidelity_rmse,
                "mean_fidelity_r2": mean_fidelity_r2,
                "std_fidelity_r2": std_fidelity_r2,
            })
    else:
        summary_stats.update({
            "mean_mae": mean_mae,
            "std_mae": std_mae,
            "mean_mse": mean_mse,
            "std_mse": std_mse,
            "mean_rmse": mean_rmse,
            "std_rmse": std_rmse,
            "mean_r2": mean_r2,
            "std_r2": std_r2,
            "mean_fidelity_mae": mean_fidelity_mae,
            "std_fidelity_mae": std_fidelity_mae,
            "mean_fidelity_mse": mean_fidelity_mse,
            "std_fidelity_mse": std_fidelity_mse,
            "mean_fidelity_rmse": mean_fidelity_rmse,
            "std_fidelity_rmse": std_fidelity_rmse,
            "mean_fidelity_r2": mean_fidelity_r2,
            "std_fidelity_r2": std_fidelity_r2,
        })

    # Load existing summary file if it exists, otherwise create new
    summary_file = os.path.join(config["data"]["results_dir_path"], "student", f"{dataset_id}_results.json")

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(summary_file), exist_ok=True)
    if os.path.exists(summary_file):
        with open(summary_file, 'r') as f:
            all_results = json.load(f)
    else:
        all_results = {}

    # Add current model results to the dataset summary
    # Use simple incremental numbering
    next_num = len(all_results) + 1
    model_key = str(next_num)

    # Add current model results to the dataset summary
    all_results[model_key] = summary_stats

    # Save updated summary
    with open(summary_file, 'w') as f:
        json.dump(all_results, f, indent=2)
    logger.info(f"Summary statistics saved to: {summary_file}")

    # -------------------------------------------------------------------------
    # SAVE STUDENT PREDICTIONS
    # -------------------------------------------------------------------------
    # Combine predictions from all outer folds
    if output_dfs:  # Check if we have any DataFrames to concatenate
        output_df = pd.concat(output_dfs, ignore_index=True)
    else:
        output_df = pd.DataFrame(columns=["index", "output"])
    output_df = output_df.sort_values(by="index")
    
    # Create the full directory structure
    sub_folder = "hpo" if use_hpo else "default"
    output_dir = os.path.join(config["data"]["output_dir_path"], sub_folder, "student")
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, f"{dataset_id}_{student_model_type}({teacher_model_type})_{model_task_type[:2]}.csv")
    output_df.to_csv(output_file, index=False)
    logger.info(f"{config['model']['student_model']} outputs saved to: {output_file}")

2025-06-06 11:38:39,611 - __main__ - INFO - Loading configuration...
2025-06-06 11:38:39,612 - __main__ - INFO - Loading dataset 23381 from cache at 'data/cache/openml_dataset_23381.pkl'...
2025-06-06 11:38:39,614 - __main__ - INFO - Dataset 23381 loaded successfully with task type: binary
2025-06-06 11:38:39,615 - __main__ - INFO - Using GPU: NVIDIA GeForce RTX 2080 Ti
2025-06-06 11:38:39,616 - __main__ - INFO - Random seed set to 42
2025-06-06 11:38:39,619 - __main__ - INFO - Loaded fold indices from: data/fold_indices/dataset_23381.json
2025-06-06 11:38:39,621 - __main__ - INFO - Loaded tabpfn outputs from: data/output/hpo/teacher/23381_tabpfn.csv
2025-06-06 11:38:39,622 - __main__ - INFO - Outer Fold 1 - Train Indices: [ 1  3  5  6  7  8  9 10 11 12], Test Indices: [ 0  2  4 18 19 27 35 39 57 59]
2025-06-06 11:38:39,623 - __main__ - INFO - -------------------- Outer Fold 1 --------------------
2025-06-06 11:38:39,624 - __main__ - INFO - Starting hyperparameter optimization for oute

2025-06-06 11:38:39,817 - __main__ - INFO - No preprocessing required for Grande model.
2025-06-06 11:38:39,818 - __main__ - INFO - Training Model on Outer Fold 1, Inner Fold 1...
/home/mherre/miniconda3/envs/thesis/lib/python3.9/site-packages/category_encoders/ordinal.py:210: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[column].fillna(-1, inplace=True)
/home/mherre/miniconda3/envs/thesis/lib/python3.9/site-packages/category_encoders/ordinal.py:210: FutureWarning: A value is trying to be set on a copy of a DataFrame or Seri

2025-06-06 11:39:20,041 - tensorflow - WARNING - 5 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7fafb856b3a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2025-06-06 11:39:20,713 - tensorflow - WARNING - 6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7fafb856b3a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2025-06-06 11:39:20,753 - __main__ - INFO - 	 Balanced Accuracy: 0.5000, F1 Score: 0.3671, ROC AUC: 0.6987
2025-06-06 11:39:20,759 - __main__ - INFO - 	 Fidelity - Balanced Accuracy: 0.5000, F1 Score: 0.4135, ROC AUC: 0.9317, KL Divergence: 0.0175